In [ ]:
#conda create --name myenv

In [1]:
conda activate mygpu


Note: you may need to restart the kernel to use updated packages.


In [2]:
import io
from google.cloud import videointelligence


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
%%writefile get_link_list.py
def lists(folder):
    import os
    import pickle
    import warnings
    warnings.filterwarnings('ignore')
    #define the path of folder containing videos
    path = "C:/Users/manas_nmr2rze/Desktop/Capstone/"+folder 
    #assign the object names to a list
    file_list=os.listdir(path)
    import re
    vid_list = []
    for string in file_list:
        m = re.search("^.*mp4$", string)
        if m:
            vid_list.append(string)
    link_list=[]
    for name in vid_list:
        name = path +"/"+ name
        link_list.append(name)
    return link_list, vid_list



Overwriting get_link_list.py


In [47]:
%%writefile links_list.py
def lists(path = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files"):
    import os
    import pickle
    import warnings
    warnings.filterwarnings('ignore')
    #define the path of folder containing videos
    #path = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files" 
    #assign the object names to a list
    file_list=os.listdir(path)
    import re
    vid_list = []
    for string in file_list:
        m = re.search("^.*mp4$", string)
        if m:
            vid_list.append(string)
    link_list=[]
    for name in vid_list:
        name = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files/"+ name
        link_list.append(name)
    return link_list, vid_list


Overwriting links_list.py


In [1]:
%%writefile links_list2.py
def lists():
    import os
    import pickle
    import warnings
    warnings.filterwarnings('ignore')
    #define the path of folder containing videos
    path = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files V2"
    #assign the object names to a list
    file_list=os.listdir(path)
    import re
    vid_list = []
    for string in file_list:
        m = re.search("^.*mp4$", string)
        if m:
            vid_list.append(string)
    link_list=[]
    for name in vid_list:
        name = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files V2/"+ name
        link_list.append(name)
    return link_list, vid_list



Overwriting links_list2.py


In [4]:
%%writefile links_list3.py
def lists():
    import os
    import pickle
    import warnings
    warnings.filterwarnings('ignore')
    #define the path of folder containing videos
    path = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files V3"
    #assign the object names to a list
    file_list=os.listdir(path)
    import re
    vid_list = []
    for string in file_list:
        m = re.search("^.*mp4$", string)
        if m:
            vid_list.append(string)
    link_list=[]
    for name in vid_list:
        name = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files V3/"+ name
        link_list.append(name)
    return link_list, vid_list



Writing links_list3.py


In [9]:
%%writefile audio_links.py
def lists():
    import os
    import pickle
    import warnings
    warnings.filterwarnings('ignore')
    #define the path of folder containing videos
    path = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/new_audio_out1/"
    #assign the object names to a list
    file_list=os.listdir(path)
    import re
    aud_list = []
    for string in file_list:
        m = re.search("^.*wav$", string)
        if m:
            aud_list.append(string)
    link_list=[]
    for name in aud_list:
        name = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/new_audio_out1/"+ name
        link_list.append(name)
    return link_list, aud_list

Overwriting audio_links.py


In [4]:
import links_list3
link_list3, vid_list3 = links_list3.lists()
len(link_list3)

99

In [5]:
video_link = (str(link_list3[12]))

In [7]:
#video_link = r"C:\Users\manas_nmr2rze\Desktop\Capstone\NI Ad Feature Extraction Project Files\ANZ Business Client 15s TVC VIEWING ONE TVC.mp4"

### Segment Detection

In [5]:
%%writefile segment_detection.py
"""Detect labels given a file path."""
def detect_shot_changes(video_link):
    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.Feature.SHOT_CHANGE_DETECTION]

    with io.open(video_link,'rb') as movie:
        input_content = movie.read()

    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content}
    )
    print("\nProcessing video for segment annotations:")

    result = operation.result().annotation_results[0]
   
    print("\nFinished processing.")
    return result
def print_video_shots(results: videointelligence.VideoAnnotationResults):
    shots = results.shot_annotations
    #shots =  results.annotation_results[0].shot_label_annotations
    print(f" Video shots: {len(shots)} ".center(40, "-"))
    for i, shot in enumerate(shots):
        t1 = shot.start_time_offset.total_seconds()
        t2 = shot.end_time_offset.total_seconds()
        print(f"{i+1:>3} | {t1:7.3f} | {t2:7.3f}")
results = detect_shot_changes(video_link)
print_video_shots(results)

Writing segment_detection.py


### Label detection

In [5]:
%%writefile video_shot_label.py
from typing import Optional, Sequence

from google.cloud import videointelligence_v1 as vi


def detect_labels(
    video_uri,
    mode: vi.LabelDetectionMode,
    segments: Optional[Sequence[vi.VideoSegment]] = None,):
    with io.open(video_uri,'rb') as movie:
        input_content = movie.read()
    video_client = vi.VideoIntelligenceServiceClient()
    features = [vi.Feature.LABEL_DETECTION]
    config = vi.LabelDetectionConfig(label_detection_mode=mode)
    context = vi.VideoContext(segments=segments, label_detection_config=config)
    operation = video_client.annotate_video(
        
        request={"features": features, "input_content": input_content, "video_context":context},
    mode = vi.LabelDetectionMode.SHOT_MODE
    )
#     request = vi.AnnotateVideoRequest(
#         input_uri=input_content,
#         features=features,
#         video_context=context,
#     )


    print(f'Processing video "{video_uri}"...')
    #operation = video_client.annotate_video(request)

    return operation.result().annotation_results[0]  # Single video
from datetime import timedelta


mode = vi.LabelDetectionMode.SHOT_MODE
segment = vi.VideoSegment(
    start_time_offset=timedelta(seconds=0),
    end_time_offset=timedelta(seconds=30),
)

#label = detect_labels(video_uri, mode, [segment])
def print_video_labels(results: vi.VideoAnnotationResults):
    labels = results.segment_label_annotations
    sort_by_first_segment_confidence(labels)

    print(f" Video labels: {len(labels)} ".center(80, "-"))
    for label in labels:
        categories = category_entities_to_str(label.category_entities)
        for segment in label.segments:
            confidence = segment.confidence
            t1 = segment.segment.start_time_offset.total_seconds()
            t2 = segment.segment.end_time_offset.total_seconds()
            print(
                f"{confidence:4.0%}",
                f"{t1:7.3f}",
                f"{t2:7.3f}",
                f"{label.entity.description}{categories}",
                sep=" | ",
            )


def sort_by_first_segment_confidence(labels: Sequence[vi.LabelAnnotation]):
    labels.sort(key=lambda label: label.segments[0].confidence, reverse=True)


def category_entities_to_str(category_entities: Sequence[vi.Entity]) -> str:
    if not category_entities:
        return ""
    entities = ", ".join([e.description for e in category_entities])
    return f" ({entities})"
#video_labels=print_video_labels(label)
def print_shot_labels(results: vi.VideoAnnotationResults):
    labels = results.shot_label_annotations
    sort_by_first_segment_start_and_confidence(labels)

    print(f" Shot labels: {len(labels)} ".center(80, "-"))
    for label in labels:
        categories = category_entities_to_str(label.category_entities)
        print(f"{label.entity.description}{categories}")
        for segment in label.segments:
            confidence = segment.confidence
            t1 = segment.segment.start_time_offset.total_seconds()
            t2 = segment.segment.end_time_offset.total_seconds()
            print(f"{confidence:4.0%} | {t1:7.3f} | {t2:7.3f}")


def sort_by_first_segment_start_and_confidence(labels: Sequence[vi.LabelAnnotation]):
    def first_segment_start_and_confidence(label):
        first_segment = label.segments[0]
        ms = first_segment.segment.start_time_offset.ToMilliseconds()
        return (ms, -first_segment.confidence)

    labels.sort(key=first_segment_start_and_confidence)
#print_shot_labels(label)

Overwriting video_shot_label.py


In [4]:
import video_shot_label
obj=video_shot_label.detect_labels(video_link[5])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (video_shot_label.py, line 17)

### Transcribe Speech (Use google cloud storage)

In [9]:
%%writefile gcloud_transcribe.py
def gcloud_transcribe(input uri):

    """Transcribe speech from a video stored on GCS."""
    from google.cloud import videointelligence

    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.Feature.SPEECH_TRANSCRIPTION]

    config = videointelligence.SpeechTranscriptionConfig(
        language_code="en-US", enable_automatic_punctuation=True
    )
    video_context = videointelligence.VideoContext(speech_transcription_config=config)

    operation = video_client.annotate_video(
        request={
            "features": features,
            "input_uri": input uri     #'gs://manas_capstone/AdelaideTourism15TVC.mp4',
            "video_context": video_context,
        }
    )

    print("\nProcessing video for speech transcription.")

    result = operation.result(timeout=600)

    # There is only one annotation_result since only
    # one video is processed.
    annotation_results = result.annotation_results[0]
    for speech_transcription in annotation_results.speech_transcriptions:

        # The number of alternatives for each transcription is limited by
        # SpeechTranscriptionConfig.max_alternatives.
        # Each alternative is a different possible transcription
        # and has its own confidence score.
        for alternative in speech_transcription.alternatives:
            print("Alternative level information:")

            print("Transcript: {}".format(alternative.transcript))
            print("Confidence: {}\n".format(alternative.confidence))

            print("Word level information:")
            for word_info in alternative.words:
                word = word_info.word
                start_time = word_info.start_time
                end_time = word_info.end_time
                print(
                    "\t{}s - {}s: {}".format(
                        start_time.seconds + start_time.microseconds * 1e-6,
                        end_time.seconds + end_time.microseconds * 1e-6,
                        word,
                    )
                )
from typing import Optional, Sequence

from google.cloud import videointelligence_v1 as vi


def transcribe_speech(
    video_uri: str,
    language_code: str,
    segments: Optional[Sequence[vi.VideoSegment]] = None):
    with io.open(video_uri,'rb') as movie:
        input_content = movie.read()

    video_client = vi.VideoIntelligenceServiceClient()
    features = [vi.Feature.SPEECH_TRANSCRIPTION]
    config = vi.SpeechTranscriptionConfig(
        language_code=language_code,
        enable_automatic_punctuation=True,
    )
    context = vi.VideoContext(
        segments=segments,
        speech_transcription_config=config,
    )
    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content, "video_context":context}
    )
#     request = vi.AnnotateVideoRequest(
#         input_uri=video_uri,
#         features=features,
#         video_context=context,
#     )

    print(f'Processing video "{video_uri}"...')
#     operation = video_client.annotate_video(request)

    return operation.result().annotation_results[0]  # Single video
def print_video_speech(results: vi.VideoAnnotationResults, min_confidence: float = 0.8):
    def keep_transcription(transcription: vi.SpeechTranscription) -> bool:
        return min_confidence <= transcription.alternatives[0].confidence

    transcriptions = results.speech_transcriptions
    transcriptions = [t for t in transcriptions if keep_transcription(t)]

    print(f" Speech Transcriptions: {len(transcriptions)} ".center(80, "-"))
    for transcription in transcriptions:
        first_alternative = transcription.alternatives[0]
        confidence = first_alternative.confidence
        transcript = first_alternative.transcript
        print(f" {confidence:4.0%} | {transcript.strip()}")
def print_word_timestamps(
    results: vi.VideoAnnotationResults, min_confidence: float = 0.8
):
    def keep_transcription(transcription: vi.SpeechTranscription) -> bool:
        return min_confidence <= transcription.alternatives[0].confidence

    transcriptions = results.speech_transcriptions
    transcriptions = [t for t in transcriptions if keep_transcription(t)]

    print(" Word Timestamps ".center(80, "-"))
    for transcription in transcriptions:
        first_alternative = transcription.alternatives[0]
        confidence = first_alternative.confidence
        for word in first_alternative.words:
            t1 = word.start_time.total_seconds()
            t2 = word.end_time.total_seconds()
            word = word.word
            print(f"{confidence:4.0%} | {t1:7.3f} | {t2:7.3f} | {word}")

Overwriting gcloud_transcribe.py


In [21]:
#From Google cloud annotation result of file stored in Googlr cloud
print_video_speech(annotation_results)

--------------------------- Speech Transcriptions: 1 ---------------------------
  91% | Winter. It's the perfect excuse to have some wine and let someone else cook book, your South Australian trip today. South Australia, rewards Wonder.


In [23]:
#print_word_timestamps(speech)

In [4]:
import detect_texts
detect_texts.detect(video_link)

Processing video "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files V3/BINGE Devils Trailer 30 TVC.mp4"...
-------------------------------- Detected Text ---------------------------------
NEW SERIES | NOW STREAMING
   96% | 0:00:00.480000 + 3.4s | 29 fr.
NEW SERIES | NOW STREAMING
   96% | 0:00:03.960000 + 4.0s | 32 fr.
NEW SERIES | NOW STREAMING
   96% | 0:00:11.040000 + 4.7s | 39 fr.
BINGE
   99% | 0:00:00 + 4.2s | 36 fr.
BINGE
   99% | 0:00:04.320000 + 3.6s | 31 fr.
BINGE
   98% | 0:00:09.960000 + 6.6s | 55 fr.
BINGE
  100% | 0:00:19.080000 + 7.8s | 66 fr.
DEVILS
  100% | 0:00:00 + 7.9s | 67 fr.
DEVILS
  100% | 0:00:09.960000 + 5.8s | 49 fr.
MA) 15+
   92% | 0:00:10.080000 + 2.6s | 23 fr.
"PATRICK DEMPSEY HAS MADE
   98% | 0:00:15.840000 + 2.0s | 18 fr.
A TRIUMPHANT RETURN"
   98% | 0:00:16.200000 + 1.7s | 15 fr.
"TWISTS
  100% | 0:00:20.520000 + 1.9s | 17 fr.


### Detect_And_Track_Text

In [15]:
%%writefile detect_texts.py
from typing import Optional, Sequence
import io
from google.cloud import videointelligence_v1 as vi


def detect_text(
    video_uri: str,
    language_hints: Optional[Sequence[str]] = None,
    segments: Optional[Sequence[vi.VideoSegment]] = None):
    with io.open(video_uri,'rb') as movie:
        input_content = movie.read()

    video_client = vi.VideoIntelligenceServiceClient()
    features = [vi.Feature.TEXT_DETECTION]
    config = vi.TextDetectionConfig(
        language_hints=language_hints,
    )
    context = vi.VideoContext(
        segments=segments,
        text_detection_config=config,
    )
    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content, "video_context":context}
    )
#     request = vi.AnnotateVideoRequest(
#         input_uri=video_uri,
#         features=features,
#         video_context=context,
#     )

    print(f'Processing video "{video_uri}"...')
#     operation = video_client.annotate_video(request)

    return operation.result().annotation_results[0]  # Single video
def detect(video_uri):
    
    from datetime import timedelta


    segment = vi.VideoSegment(
        start_time_offset=timedelta(seconds=0),
        end_time_offset=timedelta(seconds=30),
    )

    results = detect_text(video_uri, segments=[segment])
    print_video_text(results)
    return results
def print_video_text(results: vi.VideoAnnotationResults, min_frames: int = 15):
    annotations = results.text_annotations
    sort_by_first_segment_end(annotations)

    print(" Detected Text ".center(80, "-"))
    for annotation in annotations:
        for text_segment in annotation.segments:
            frames = len(text_segment.frames)
            if frames < min_frames:
                continue
            text = annotation.text
            confidence = text_segment.confidence
            start = text_segment.segment.start_time_offset
            seconds = segment_seconds(text_segment.segment)
            print(text)
            print(f"  {confidence:4.0%} | {start} + {seconds:.1f}s | {frames} fr.")


def sort_by_first_segment_end(annotations: Sequence[vi.TextAnnotation]):
    def first_segment_end(annotation):
        return annotation.segments[0].segment.end_time_offset.ToMilliseconds()

    annotations.sort(key=first_segment_end)


def segment_seconds(segment: vi.VideoSegment) -> float:
    t1 = segment.start_time_offset.total_seconds()
    t2 = segment.end_time_offset.total_seconds()
    return t2 - t1
def video_text(results: vi.VideoAnnotationResults, min_frames: int = 15):
    annotations = results.text_annotations
    sort_by_first_segment_end(annotations)
    text_list = []

    print(" Detected Text ".center(80, "-"))
    for annotation in annotations:
        for text_segment in annotation.segments:
            frames = len(text_segment.frames)
            if frames < min_frames:
                continue
            text = annotation.text
            confidence = text_segment.confidence
            start = text_segment.segment.start_time_offset
            seconds = segment_seconds(text_segment.segment)
            print(text)
            #print(f"  {confidence:4.0%} | {start} + {seconds:.1f}s | {frames} fr.")
            text_list.append(text)
    return text_list


def sort_by_first_segment_end(annotations: Sequence[vi.TextAnnotation]):
    def first_segment_end(annotation):
        return annotation.segments[0].segment.end_time_offset.ToMilliseconds()

    annotations.sort(key=first_segment_end)


def segment_seconds(segment: vi.VideoSegment) -> float:
    t1 = segment.start_time_offset.total_seconds()
    t2 = segment.end_time_offset.total_seconds()
    return t2 - t1
def print_text_frames(results: vi.VideoAnnotationResults, contained_text: str):
    # Vertex order: top-left, top-right, bottom-right, bottom-left
    def box_top_left(box: vi.NormalizedBoundingPoly) -> str:
        tl = box.vertices[0]
        return f"({tl.x:.5f}, {tl.y:.5f})"

    def box_bottom_right(box: vi.NormalizedBoundingPoly) -> str:
        br = box.vertices[2]
        return f"({br.x:.5f}, {br.y:.5f})"

    annotations = results.text_annotations
    annotations = [a for a in annotations if contained_text in a.text]
    for annotation in annotations:
        print(f" {annotation.text} ".center(80, "-"))
        for text_segment in annotation.segments:
            for frame in text_segment.frames:
                frame_ms = frame.time_offset.total_seconds()
                box = frame.rotated_bounding_box
                print(
                    f"{frame_ms:>7.3f}",
                    box_top_left(box),
                    box_bottom_right(box),
                    sep=" | ",
                )
                

Overwriting detect_texts.py


In [26]:
text_results

segment {
  start_time_offset {
  }
  end_time_offset {
    seconds: 14
    nanos: 960000000
  }
}
text_annotations {
  text: "M"
  segments {
    segment {
      start_time_offset {
        seconds: 11
        nanos: 400000000
      }
      end_time_offset {
        seconds: 11
        nanos: 520000000
      }
    }
    confidence: 0.804432631
    frames {
      rotated_bounding_box {
        vertices {
          x: 0.584375
          y: 0.759259284
        }
        vertices {
          x: 0.608271241
          y: 0.762323439
        }
        vertices {
          x: 0.606884897
          y: 0.796493948
        }
        vertices {
          x: 0.58298862
          y: 0.793429732
        }
      }
      time_offset {
        seconds: 11
        nanos: 400000000
      }
    }
    frames {
      rotated_bounding_box {
        vertices {
          x: 0.584375
          y: 0.758333325
        }
        vertices {
          x: 0.607200503
          y: 0.761964
        }
        vertices {

In [28]:
print_video_text(text_results)

-------------------------------- Detected Text ---------------------------------
Start collecting today.
   99% | 0:00:12.960000 + 1.9s | 17 fr.
Hurry in-store.
   99% | 0:00:12.960000 + 1.9s | 17 fr.
Australia Post
  100% | 0:00:12.960000 + 1.9s | 17 fr.
#Great AussieCoinHunt
   96% | 0:00:12.960000 + 1.9s | 17 fr.
Available at participating Post Offices.
   99% | 0:00:12.960000 + 1.9s | 17 fr.


In [30]:
vid_text=video_text(text_results)

-------------------------------- Detected Text ---------------------------------
Start collecting today.
Hurry in-store.
Australia Post
#Great AussieCoinHunt
Available at participating Post Offices.


In [32]:
contained_text = "Aussie"
print_text_frames(text_results, contained_text)

---------------------------- #Great AussieCoinHunt -----------------------------
 12.960 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.080 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.200 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.320 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.440 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.560 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.680 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.800 | (0.41146, 0.82593) | (0.59219, 0.86019)
 13.920 | (0.41146, 0.82593) | (0.59219, 0.86019)
 14.040 | (0.41042, 0.82778) | (0.59375, 0.86019)
 14.160 | (0.41146, 0.82593) | (0.59219, 0.86019)
 14.280 | (0.41146, 0.82593) | (0.59219, 0.86019)
 14.400 | (0.41146, 0.82593) | (0.59219, 0.86019)
 14.520 | (0.41146, 0.82593) | (0.59219, 0.86019)
 14.640 | (0.41250, 0.82778) | (0.59167, 0.86019)
 14.760 | (0.41146, 0.82593) | (0.59219, 0.86019)
 14.880 | (0.41250, 0.82778) | (0.59167, 0.86019)


### Detect and Track Objects

In [9]:
%%writefile detect_object.py
from typing import Optional, Sequence

from google.cloud import videointelligence_v1 as vi
import io

def track_objects(
    video_uri: str, segments: Optional[Sequence[vi.VideoSegment]] = None):
    import io
    with io.open(video_uri,'rb') as movie:
        input_content = movie.read()

    video_client = vi.VideoIntelligenceServiceClient()
    features = [vi.Feature.OBJECT_TRACKING]
    context = vi.VideoContext(segments=segments)
    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content, "video_context":context}
    )
#     request = vi.AnnotateVideoRequest(
#         input_uri=video_uri,
#         features=features,
#         video_context=context,
#     )

    print(f'Processing video "{video_uri}"...')
#     operation = video_client.annotate_video(request)

    return operation.result().annotation_results[0]  # Single video
from datetime import timedelta


segment = vi.VideoSegment(
    start_time_offset=timedelta(seconds=0),
    end_time_offset=timedelta(seconds=30),
)

#objects = track_objects(video_uri, [segment])
def print_detected_objects(
    results: vi.VideoAnnotationResults, min_confidence: float = 0.0):
    annotations = results.object_annotations
    annotations = [a for a in annotations if min_confidence <= a.confidence]

    print(
        f" Detected objects: {len(annotations)}"
        f" ({min_confidence:.0%} <= confidence) ".center(80, "-")
    )
    entity_list=[]
    for annotation in annotations:
        entity = annotation.entity
        description = entity.description
        entity_id = entity.entity_id
        confidence = annotation.confidence
        t1 = annotation.segment.start_time_offset.total_seconds()
        t2 = annotation.segment.end_time_offset.total_seconds()
        frames = len(annotation.frames)
        entity_list.append((description,t1,t2,frames))
#         print(
#             f"{description:<22}",
#             f"{entity_id:<10}",
#             f"{confidence:4.0%}",
#             f"{t1:>7.3f}",
#             f"{t2:>7.3f}",
#             f"{frames:>2} fr.",
#             sep=" | ",
# )
    item_set=set([i[0] for i in entity_list])
    items = list(item_set)
    item_dir={}
    for i in items:
        item_dir[i]=[]
    for j, item in enumerate(entity_list):
        if len(item_dir[item[0]])> 0:
            item_dir[item[0]].append((entity_list[j][1],entity_list[j][2]))
            item_dir[item[0]][0]+=entity_list[j][3]
        else:
            item_dir[item[0]].append(entity_list[j][3])
            item_dir[item[0]].append((entity_list[j][1],entity_list[j][2]))
    
    item_dict={}
    for key, item in item_dir.items():
        item_dict[key]={'Total frames':item_dir[key][0], 'Timestamps':tuple(item_dir[key][1:])}
        
        

            

        
        
    return items, item_dict
def print_object_frames(
    results: vi.VideoAnnotationResults, entity_id: str, min_confidence: float = 0.0
):
    def keep_annotation(annotation: vi.ObjectTrackingAnnotation) -> bool:
        return all(
            [
                annotation.entity.entity_id == entity_id,
                min_confidence <= annotation.confidence,
            ]
        )

    annotations = results.object_annotations
    annotations = [a for a in annotations if keep_annotation(a)]
    for annotation in annotations:
        description = annotation.entity.description
        confidence = annotation.confidence
        print(
            f" {description},"
            f" confidence: {confidence:.0%},"
            f" frames: {len(annotation.frames)} ".center(80, "-")
        )
        for frame in annotation.frames:
            t = frame.time_offset.total_seconds()
            box = frame.normalized_bounding_box
            print(
                f"{t:>7.3f}",
                f"({box.left:.5f}, {box.top:.5f})",
                f"({box.right:.5f}, {box.bottom:.5f})",
                sep=" | ",
            )  

Overwriting detect_object.py


In [6]:
import detect_object
objects = detect_object.track_objects(video_link)
items,item_dict=detect_object.print_detected_objects(objects)

Processing video "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files V3/BINGE Devils Trailer 30 TVC.mp4"...
------------------- Detected objects: 108 (0% <= confidence) -------------------


In [8]:
item_dict

{'building': {'Total frames': 92,
  'Timestamps': ((4.08, 4.68),
   (15.84, 17.88),
   (15.84, 17.76),
   (15.84, 17.88),
   (15.84, 17.52),
   (15.84, 17.88))},
 'outerwear': {'Total frames': 55,
  'Timestamps': ((5.4, 5.64),
   (10.08, 12.24),
   (10.08, 10.8),
   (13.8, 15.72),
   (21.48, 22.44))},
 'shirt': {'Total frames': 7, 'Timestamps': ((2.4, 2.52), (12.24, 12.72))},
 'suit': {'Total frames': 36,
  'Timestamps': ((2.64, 3.24), (15.84, 17.88), (19.08, 20.4))},
 'glasses': {'Total frames': 1, 'Timestamps': ((13.8, 13.8),)},
 'lighting': {'Total frames': 1, 'Timestamps': ((1.8, 1.8),)},
 'license plate': {'Total frames': 1, 'Timestamps': ((27.12, 27.12),)},
 'clothing': {'Total frames': 61,
  'Timestamps': ((1.68, 1.68),
   (3.48, 3.84),
   (4.8, 5.04),
   (4.92, 5.04),
   (5.04, 5.04),
   (5.64, 5.64),
   (5.88, 7.68),
   (22.56, 23.16),
   (22.56, 23.16),
   (23.4, 24.12),
   (23.4, 24.24),
   (28.08, 28.56),
   (28.08, 28.08))},
 'television': {'Total frames': 3, 'Timestamps':

In [8]:
entity_list

[('person', 0.0, 1.68, 15),
 ('person', 0.0, 0.24, 3),
 ('tie', 0.24, 0.84, 6),
 ('person', 0.36, 0.6, 3),
 ('person', 1.44, 1.68, 3),
 ('clothing', 1.68, 1.68, 1),
 ('lighting', 1.8, 1.8, 1),
 ('tie', 1.92, 2.52, 6),
 ('person', 2.04, 2.52, 5),
 ('shirt', 2.4, 2.52, 2),
 ('person', 2.64, 3.24, 6),
 ('suit', 2.64, 3.24, 6),
 ('person', 2.88, 3.24, 4),
 ('person', 3.48, 3.96, 5),
 ('clothing', 3.48, 3.84, 4),
 ('person', 4.08, 4.68, 6),
 ('car', 4.08, 4.68, 6),
 ('building', 4.08, 4.68, 6),
 ('car', 4.32, 4.68, 4),
 ('car', 4.32, 4.68, 4),
 ('car', 4.32, 4.68, 4),
 ('car', 4.32, 4.68, 4),
 ('car', 4.32, 4.68, 4),
 ('person', 4.32, 4.68, 4),
 ('person', 4.68, 4.68, 1),
 ('person', 4.8, 5.04, 3),
 ('person', 4.8, 5.04, 3),
 ('person', 4.8, 5.04, 3),
 ('person', 4.8, 5.04, 3),
 ('person', 4.8, 5.04, 3),
 ('person', 4.8, 5.04, 3),
 ('person', 4.8, 5.04, 3),
 ('clothing', 4.8, 5.04, 3),
 ('clothing', 4.92, 5.04, 2),
 ('person', 5.04, 5.04, 1),
 ('clothing', 5.04, 5.04, 1),
 ('person', 5.16, 

In [38]:
entity_id = "/m/0jbk"
print_object_frames(objects, entity_id)

---------------------- animal, confidence: 69%, frames: 1 ----------------------
  2.880 | (0.44627, 0.14896) | (0.65265, 0.41929)
---------------------- animal, confidence: 69%, frames: 1 ----------------------
  3.600 | (0.64190, 0.01986) | (0.85319, 0.24271)
---------------------- animal, confidence: 66%, frames: 1 ----------------------
  6.600 | (0.08063, 0.41238) | (0.40772, 0.99978)
---------------------- animal, confidence: 70%, frames: 1 ----------------------
  8.160 | (0.22983, 0.64879) | (0.50856, 0.87668)
---------------------- animal, confidence: 67%, frames: 1 ----------------------
 10.080 | (0.14504, 0.32142) | (0.49778, 0.84025)


### Detect and Track Logos

In [23]:
%%writefile detect_logo.py
from typing import Optional, Sequence

from google.cloud import videointelligence_v1 as vi
def logo_and_frames(video):
    detect_logos(video)

    results = detect_logos(video, [segment])
    entity = print_detected_logos(results)
    detect_logo(local_file_path=video)
    #print(detect_logos(video))

def detect_logos(
    video_uri: str, segments: Optional[Sequence[vi.VideoSegment]] = None):
    with io.open(video_uri,'rb') as movie:
        input_content = movie.read()

    video_client = vi.VideoIntelligenceServiceClient()
    features = [vi.Feature.LOGO_RECOGNITION]
    context = vi.VideoContext(segments=segments)
    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content, "video_context":context}
    )
#     request = vi.AnnotateVideoRequest(
#         input_uri=video_uri,
#         features=features,
#         video_context=context,
#     )

    print(f'Processing video "{video_uri}"...')
#     operation = video_client.annotate_video(request)

    return operation.result().annotation_results[0]  # Single video
from datetime import timedelta


segment = vi.VideoSegment(
    start_time_offset=timedelta(seconds=0),
    end_time_offset=timedelta(seconds=30),
)

#results = detect_logos(video_uri, [segment])  
def print_detected_logos(results: vi.VideoAnnotationResults):
    annotations = results.logo_recognition_annotations
    frame_dict = {}
    frame_list=[]
    entity_list=[]
    logo_timestamp=[]


    print(f" Detected logos: {len(annotations)} ".center(80, "-"))
    for annotation in annotations:
        entity = annotation.entity
        entity_id = entity.entity_id
        description = entity.description
        frame_dict[description]=0
        for track in annotation.tracks:
            confidence = track.confidence
            t1 = track.segment.start_time_offset.total_seconds()
            t2 = track.segment.end_time_offset.total_seconds()
            logo_frames = len(track.timestamped_objects)
            frame_dict[description]+=len(track.timestamped_objects)
            logo_timestamp.append((description,track.segment.start_time_offset.total_seconds(),
                                   track.segment.end_time_offset.total_seconds()))
            print(
                f"{confidence:4.0%}",
                f"{t1:>7.3f}",
                f"{t2:>7.3f}",
                f"{logo_frames:>3} fr.",
                f"{entity_id:<15}",
                f"{description}",                
                sep=" | ",
            )
    for key, value in frame_dict.items():
        entity_list.append(key)
        frame_list.append(value)
    max_frame = max(frame_list)
    max_frame_entity = entity_list[frame_list.index(max(frame_list))]
    print(f"Logo with max frames: {max_frame_entity}")
    
    print("Total frames with","'",max_frame_entity,"'", f"logo: {max_frame}")
    sorted_frame_list, sorted_frame_dict=create_entity_output(frame_dict)
    return sorted_frame_list, sorted_frame_dict, logo_timestamp
            
#f"{area_dict[entity_id]}",
def print_logo_frames(results: vi.VideoAnnotationResults, entity_id: str):
    def keep_annotation(annotation: vi.LogoRecognitionAnnotation) -> bool:
        return annotation.entity.entity_id == entity_id

    annotations = results.logo_recognition_annotations
    annotations = [a for a in annotations if keep_annotation(a)]
    for annotation in annotations:
        description = annotation.entity.description
        for track in annotation.tracks:
            confidence = track.confidence
            print(
                f" {description},"
                f" confidence: {confidence:.4%},"
                f" frames: {len(track.timestamped_objects)} ".center(80, "-")
            )
            for timestamped_object in track.timestamped_objects:
                t = timestamped_object.time_offset.total_seconds()
                box = timestamped_object.normalized_bounding_box
                print(
                    f"{t:>7.3f}",
                    f"({box.left:.5f}, {box.top:.5f})",
                    f"({box.right:.5f}, {box.bottom:.5f})",
                    sep=" | ",
                )
import io

from google.cloud import videointelligence


def detect_logo(local_file_path):
    from statistics import mean
    from collections import Counter
    """Performs asynchronous video annotation for logo recognition on a local file."""

    client = videointelligence.VideoIntelligenceServiceClient()

    with io.open(local_file_path, "rb") as f:
        input_content = f.read()
    features = [videointelligence.Feature.LOGO_RECOGNITION]

    operation = client.annotate_video(
        request={"features": features, "input_content": input_content}
    )

    print("Waiting for operation to complete...")
    response = operation.result()

    # Get the first response, since we sent only one video.
    annotation_result = response.annotation_results[0]
    ent_list = []
    ent_dir={}
    ent_dir_val={}
    frame_dict={}
    area_list=[]
    ent_id_list=[]
    logo_pos_dict,logo_dict_out ={},{}
    logo_pos_list=[]
    max_logo_list=[]
    empty_list=[]
    empty_list2=[]
    logo_timestamp_list=[]
    logo_position={1:"Top left", 2:"Left center", 3:"Lower left",4:"Bottom center",5:"Lower right",
                  6:"Right center",7:"Top right",8:"Top center", 9:"Center of screen", 10:"Not sure"}

    # Annotations for list of logos detected, tracked and recognized in video.
    for  logo_recognition_annotation in annotation_result.logo_recognition_annotations:
        entity = logo_recognition_annotation.entity

        # Opaque entity ID. Some IDs may be available in [Google Knowledge Graph
        # Search API](https://developers.google.com/knowledge-graph/).
        print("Entity Id : {}".format(entity.entity_id))

        print("Description : {}".format(entity.description))
        
        ent_list.append(entity.description)
        frame_dict[entity.description] = 0
        ent_id_list.append(entity.entity_id)
        

        # All logo tracks where the recognized logo appears. Each track corresponds
        # to one logo instance appearing in consecutive frames.
        for j, track in enumerate(logo_recognition_annotation.tracks):
            ent_dir[f"{entity.description}_{j}"]=(f"Start time:{(float(track.segment.start_time_offset.seconds))+(float(track.segment.start_time_offset.microseconds)/1000000)}",
                                         f"End time: {(float(track.segment.end_time_offset.seconds)+float(track.segment.end_time_offset.microseconds/1000000))}",
                                         f"Duration of logo: {round(((float(track.segment.end_time_offset.seconds))+(float(track.segment.end_time_offset.microseconds)/1000000))-((float(track.segment.start_time_offset.seconds))+(float(track.segment.start_time_offset.microseconds)/1000000)),3)}",
                                         f"Confidence: {round(track.confidence*100,2)}%")
            ent_dir_val[entity.description]=(((float(track.segment.start_time_offset.seconds))+(float(track.segment.start_time_offset.microseconds)/1000000),
                                         (float(track.segment.end_time_offset.seconds)+float(track.segment.end_time_offset.microseconds/1000000)),
                                         round(((float(track.segment.end_time_offset.seconds))+(float(track.segment.end_time_offset.microseconds)/1000000))-((float(track.segment.start_time_offset.seconds))+(float(track.segment.start_time_offset.microseconds)/1000000)),3)))
            logo_pos_dict[entity.description]=0
#             max_logo_dict[entity.description]=0
            x=0
            logo_pos_lst=[]
            ctr={}
            # Video segment of a track.
            print(
                "\n\tStart Time Offset : {}.{}".format(
                    track.segment.start_time_offset.seconds,
                    track.segment.start_time_offset.microseconds * 1000,
                )
            )
            print(
                "\tEnd Time Offset : {}.{}".format(
                    track.segment.end_time_offset.seconds,
                    track.segment.end_time_offset.microseconds * 1000,
                )
            )
#             print("\tConfidence : {}".format(track.confidence))
#             print(f" Area covered: {mean(area_lst)}")
#             print(f" frames: {len(track.timestamped_objects)} ".center(80, "-"))
            
            if frame_dict[entity.description] is not None:
                
                frame_dict[entity.description] = frame_dict[entity.description]+len(track.timestamped_objects)
            else:
                frame_dict[entity.description]= len(track.timestamped_objects)
            
            area_lst = []
            for timestamped_object in track.timestamped_objects:

                # Normalized Bounding box in a frame, where the object is located.
                normalized_bounding_box = timestamped_object.normalized_bounding_box
                Area=(float(normalized_bounding_box.bottom)-float(normalized_bounding_box.top))*(float(normalized_bounding_box.right)-float(normalized_bounding_box.left))*100
                area_lst.append(Area)
                
            area_list.append((entity.description, f"{round(mean(area_lst),4)}%"))    
            print("\tConfidence : {}".format(track.confidence))
            print(f"\tAverage area covered: {round(mean(area_lst),4)}%")
            
            for timestamped_object in track.timestamped_objects:
                normalized_bounding_box = timestamped_object.normalized_bounding_box                
                position=detect_position(normalized_bounding_box.top,normalized_bounding_box.left,normalized_bounding_box.bottom,normalized_bounding_box.right)
                logo_pos_lst.append(logo_position[position])
                #area_lst.append(Area)
            
            for i in logo_pos_lst:
                ctr[i]=logo_pos_lst.count(i)
            for k, v in ctr.items():
                if v>x:
                    x=v
                    max_appear=k
                    count=x
                else:
                    continue
            
#             if logo_pos_dict[entity.description] != None:
            max_logo_list.append((entity.description,max_appear, count))
#             if max_logo_dict[max_appear] != None:                
#                 max_logo_dict[max_appear]+=logo_pos_dict[max_appear]
            print(f"\tMax appeared logo position: {max_appear} ")
            print(f"\tframes: {len(track.timestamped_objects)} ".center(80, "-"))
                

            
            
            # The object with timestamp and attributes per frame in the track.

            logo = entity.description
            for i,timestamped_object in enumerate(track.timestamped_objects):

#                 Normalized Bounding box in a frame, where the object is located.
                normalized_bounding_box = timestamped_object.normalized_bounding_box
                Area=(float(normalized_bounding_box.bottom)-float(normalized_bounding_box.top))*(float(normalized_bounding_box.right)-float(normalized_bounding_box.left))*100
                position=detect_position(normalized_bounding_box.top,normalized_bounding_box.left,normalized_bounding_box.bottom,normalized_bounding_box.right)
                print("\n\t\tLeft : {}".format(normalized_bounding_box.left))
                print("\t\tTop : {}".format(normalized_bounding_box.top))
                print("\t\tRight : {}".format(normalized_bounding_box.right))
                print("\t\tBottom : {}".format(normalized_bounding_box.bottom))
                print("\t\tPercentage of screen covered :{}%".format(round(Area,3)))
                print("\t\tLogo position :{}".format(logo_position[position]))
                logo_pos_lst.append(logo_position[position])
                #area_lst.append(Area)
                logo_dict_out[f'{logo}_{j}_{i}']={"Logo ":logo,
                                                                "Percentage of screen covered ":f"{(round(Area,3))}%",
                                                                "Logo position ":(logo_position[position])}

            


            logo_out={}
            for key, item in ent_dir.items():
                lst=[]
                str=''
                cont=''
                for m, n in logo_dict_out.items():
                    if key in m:
                        lst.append(n)
                for i in item:
                    str=str+i+','
               
                    
                logo_out[str]=tuple(lst)
                    
                    
                

                # Optional. The attributes of the object in the bounding box.
                for attribute in timestamped_object.attributes:
                    print("\n\t\t\tName : {}".format(attribute.name))
                    print("\t\t\tConfidence : {}".format(attribute.confidence))
                    print("\t\t\tValue : {}".format(attribute.value))

            # Optional. Attributes in the track level.
            for track_attribute in track.attributes:
                print("\n\t\tName : {}".format(track_attribute.name))
                print("\t\tConfidence : {}".format(track_attribute.confidence))
                print("\t\tValue : {}".format(track_attribute.value))

        # All video segments where the recognized logo appears. There might be
        # multiple instances of the same logo class appearing in one VideoSegment.
        for segment in logo_recognition_annotation.segments:
            logo_timestamp_list.append((entity.description,{segment.start_time_offset.seconds,
                    segment.start_time_offset.microseconds * 1000,},{segment.end_time_offset.seconds,
                    segment.end_time_offset.microseconds * 1000,}))
            print(
                "\n\tStart Time Offset : {}.{}".format(
                    segment.start_time_offset.seconds,
                    segment.start_time_offset.microseconds * 1000,
                )
            )
            print(
                "\tEnd Time Offset : {}.{}".format(
                    segment.end_time_offset.seconds,
                    segment.end_time_offset.microseconds * 1000,
                )
                
            )
    return area_list, max_logo_list,logo_out

def detect_position(top, left, bottom, right):
    try:
        if top<.5 and left<.5 and bottom<=.5 and right<=.5:
            return 1
        if top<=.5 and left<.5 and bottom>=.5 and right<=.5:
            return 2
        if top>=.5 and left<.5 and bottom>.5 and right<=.5:
            return 3
        if top>=.5 and left<.5 and bottom>.5 and right>.5:
            return 4
        if top>=.5 and left>=.5 and bottom>.5 and right>.5:
            return 5
        if top<=.5 and left>=.5 and bottom>.5 and right>.5:
            return 6
        if top<.5 and left>=.5 and bottom<=.5 and right>.5:
            return 7
        if top<.5 and left<=.5 and bottom<=.5 and right>.5:
            return 8
        if top<.5 and left<.5 and bottom>.5 and right>.5:
            return 9
    except:
        return 10
#     if len(ent_list)>=1:
#         max=0
#         ide =''
#         max_dic ={}
#         for key, value in ent_dir_val.items():
            
#             if value[2]>max:
#                 max=value[2]
#                 ide = key
#         max_dic[ide] = max
                
def create_entity_output(frame_dict):
    sorted_frame_list = sorted(frame_dict.items(), key=lambda x:x[1],reverse=True)
    sorted_frame_dict = dict(sorted_frame_list)
    if len(sorted_frame_list)>5:
        sorted_frame_list=sorted_frame_list[:5]
    while len(sorted_frame_list)<5:
        sorted_frame_list.append((0,0))
    return sorted_frame_list, sorted_frame_dict
        
        
        
        
    
        
    
            
            
        
#         print(f"Logos found in video: {ent_list}")
#         print(f"Frames per logo in video: {frame_dict}")
#         print(f"Max duaration in video : Logo: {ide}, Duration: {max}")
#         print(f"First logo in video: {ent_list[0]} : {ent_dir[ent_list[0]]}")
#         print(f"Last logo in video: {ent_list[-1]} : {ent_dir[ent_list[-1]]}")
#     else:
#         pass
        

Overwriting detect_logo.py


In [14]:
import detect_logo
detect_logo.detect_logos(video_link)

NameError: name 'video_link' is not defined

In [ ]:
entity = print_detected_logos(results)

In [ ]:
entity_id = "/m/02xnhc"
print_object_frames(results, entity_id)

In [15]:
from statistics import mean
l =[1,2,3,5,6]
mean(l)

3.4

In [16]:
maps_entity_id = "/g/11g1mklvr2"
print_logo_frames(results, maps_entity_id)

NameError: name 'print_logo_frames' is not defined

### Detect Logo

In [48]:
detect_logo(local_file_path=video_link)

Waiting for operation to complete...
Entity Id : /m/0262n_w
Description : Lysol

	Start Time Offset : 11.640000000
	End Time Offset : 12.840000000
	Confidence : 0.9115594625473022

		Left : 0.8374731540679932
		Top : 0.919912576675415
		Right : 0.9549275636672974
		Bottom : 0.9984924793243408

		Left : 0.8475931882858276
		Top : 0.8984063863754272
		Right : 0.9473701119422913
		Bottom : 0.9978735446929932

		Left : 0.8506121635437012
		Top : 0.9026666879653931
		Right : 0.9398094415664673
		Bottom : 0.9814058542251587

		Left : 0.8347606658935547
		Top : 0.8711968064308167
		Right : 0.9412937760353088
		Bottom : 0.9777390956878662

		Left : 0.8170974850654602
		Top : 0.8551164865493774
		Right : 0.941670298576355
		Bottom : 0.9872922897338867

		Left : 0.811427116394043
		Top : 0.8466477990150452
		Right : 0.939243733882904
		Bottom : 0.9832173585891724

		Left : 0.8068070411682129
		Top : 0.8372074365615845
		Right : 0.94095379114151
		Bottom : 0.9780781865119934

		Left : 0.807903230

In [4]:
%%writefile logo_df_generator.py
def generate_logo_dataframe(video_link,df,col_list):
    #generate empty dataframe and pass it the function
    import detect_logo
    try:
        area_list, max_logo_list, logo_out=detect_logo.detect_logo(video_link)
    except:
        return "Logo detection is not possible..."
    try:
        results = detect_logo.detect_logos(video_link)
        sorted_frame_list, sorted_frame_dict, logo_timestamp = detect_logo.print_detected_logos(results)
    except:
        return "Logo not printed"
    try:
    
        avg_area_dict=avg_area_logo(area_list)
        max_logo_pos_dict=max_appeared_logo_pos(max_logo_list)
    except:
        return "Average area and max appeared position counting is not possible"
    try:
        
        logo_dataframe=add_data(video_link,avg_area_dict,max_logo_pos_dict,sorted_frame_dict,sorted_frame_list, df, col_list)
        return logo_dataframe, sorted_frame_list, sorted_frame_dict,avg_area_dict,max_logo_pos_dict, logo_timestamp,logo_out
    except:
        pass


def make_empty_df():
    import pandas as pd

    df = pd.DataFrame(columns=['logo_1_total_frames','logo_2_total_frames','logo_3_total_frames','logo_4_total_frames',
                               'logo_5_total_frames','logo_1_avg_logo_size','logo_2_avg_logo_size','logo_3_avg_logo_size',
                               'logo_4_avg_logo_size','logo_5_avg_logo_size','logo_1_most_appeared_position',
                               'logo_2_most_appeared_position','logo_3_most_appeared_position','logo_4_most_appeared_position',
                               'logo_5_most_appeared_position','video','logos'])
    col_list=[df.columns[0], df.columns[1],df.columns[2],
          df.columns[3],df.columns[4],df.columns[5],
          df.columns[6],df.columns[7],df.columns[8],
          df.columns[9],df.columns[10],df.columns[11],
          df.columns[12],df.columns[13],df.columns[14],df.columns[15],df.columns[16]]
    
    return df, col_list    
    
    
    
def avg_area_logo(area_list):# first output from detect logo function
    area_set=set()
    avg_area_dict={}
    for i, x in enumerate(area_list):
        area_set.add(x[0])
    for i, x in enumerate(area_set):
        avg_area_dict[x]=0
        c=0
        for j in area_list:
            if x==j[0]:
                c+=1
                avg_area_dict[x]+= float(j[1].replace('%',''))
        avg_area_dict[x]= avg_area_dict[x]/c
    return avg_area_dict

def max_appeared_logo_pos(max_logo_list): # second output from detect logo function
    max_logo_set=set()
    max_logo_pos_dict={}
    for i, x in enumerate(max_logo_list):
        max_logo_set.add(x[0])
    for i, x in enumerate(max_logo_set):
        max_logo_pos_dict[x]=[]
        c=0
        for j in max_logo_list:
            if x==j[0]:
                max_logo_pos_dict[x].append(j[1])
        max_logo_pos_dict[x]= max_count(max_logo_pos_dict[x])
    return max_logo_pos_dict
def max_count(list_name):
    from collections import Counter
    counter = Counter(list_name)
    max_count_object = counter.most_common(1)
    max_count_object = max_count_object[0][0]
    return max_count_object



def add_data(video_link,avg_area_dict,max_logo_pos_dict,sorted_frame_dict,sorted_frame_list, df, col_list):

    import pandas as pd
    item_dic={}
    item_list=[]
    for i in range(len(sorted_frame_dict)):
        item_list.append(sorted_frame_dict[sorted_frame_list[i][0]])
        item_list.append(avg_area_dict[sorted_frame_list[i][0]])
        item_list.append(max_logo_pos_dict[sorted_frame_list[i][0]])
 
    j=0
    for i in range(len(sorted_frame_dict)):    
        item_dic[col_list[i]]=item_list[j]
        item_dic[col_list[i+5]]=item_list[j+1]
        item_dic[col_list[i+10]]=item_list[j+2]
        j+=3
    item_dic[col_list[15]]= video_link
    item_dic[col_list[16]]=[key for key, value in sorted_frame_dict.items()]
    new_row=[item_dic]

    df = df.append(new_row, ignore_index=True)
    return df

def predict_logo(video_link):
   
    import pandas as pd
    from sklearn.ensemble import GradientBoostingClassifier
    import pickle
    rf_model = pickle.load(open('rf_model.pkl','rb'))
    df,col_list= make_empty_df()    
    logo_df, sorted_frame_list, sorted_frame_dict,avg_area_dict,max_logo_pos_dict, logo_timestamp,logo_out=generate_logo_dataframe(video_link,df,col_list)
    logo_position={"Top left":1, "Left center":2, "Lower left":3,"Bottom center":4,"Lower right":5,
              "Right center":6,"Top right":7,"Top center":8, "Center of screen":9, "Not sure":10}
    logo_df['logo_1_most_appeared_position']=logo_df['logo_1_most_appeared_position'].map(logo_position)
    logo_df['logo_2_most_appeared_position']=logo_df['logo_2_most_appeared_position'].map(logo_position)
    logo_df['logo_3_most_appeared_position']=logo_df['logo_3_most_appeared_position'].map(logo_position)
    logo_df['logo_4_most_appeared_position']=logo_df['logo_4_most_appeared_position'].map(logo_position)
    logo_df['logo_5_most_appeared_position']=logo_df['logo_5_most_appeared_position'].map(logo_position)
    logo_df = logo_df.drop(columns=['video', 'logos'], axis=1)
    logo_df= logo_df.fillna(0)
    logo_out_dict={}
    timestamps=[]
    timestamps1=[]
    
    predict = rf_model.predict(logo_df.iloc[0:1])
    index= int(predict[0])-1
    logo = sorted_frame_list[index][0]
    for i , x in enumerate(logo_timestamp):
        if logo_timestamp[i][0]== logo:
            timestamps.append((logo_timestamp[i][1], logo_timestamp[i][2]))
    try:

        logo_out_dict['Logo'] = logo
    except:
        pass
    try:
        logo_out_dict['Total_frames'] = sorted_frame_dict[logo]
    except:
        pass
    try:
        logo_out_dict['Avg_area'] = avg_area_dict[logo]
    except:
        pass
    try:
        logo_out_dict['Max_appeared_position'] = max_logo_pos_dict[logo]
    except:
        pass
    try:
  
        logo_out_dict['Timestamps']= timestamps
    except:
        pass
  
    

    return logo_out_dict,logo_out,logo

    
    
    
    
 

Overwriting logo_df_generator.py


In [5]:
%%writefile create_json.py
import json

def my_logo_json(video_uri):
    import logo_df_generator
    #area_list, max_logo_list, logo_out=detect_logo.detect_logo(video_uri)
    logo_out_dict,logo_out,logo=logo_df_generator.predict_logo(video_uri)
    my_dict = logo_out
    my_logo = logo_out_dict
    my_json_out_dict = json.dumps(my_dict)
    my_json_out_sum = json.dumps(my_logo)
    
    return my_json_out_sum, my_json_out_dict, logo

def create_json_logo_file(video_uri,path='C:/Users/manas_nmr2rze/Desktop/Capstone/Output/JSON_logo/' ):
    import logo_df_generator, sys
    my_json_out_sum, my_json_out_dict,logo=my_logo_json(video_uri)
    filepath_1=path+logo+'_Detail.json'
    filepath_2=path+logo+'_Summary.json'
    
    with open(filepath_1, "w") as outfile:
        sys.stdout = filepath_1
        outfile.write(my_json_out_sum)
    with open(filepath_2, "w") as outfile:
        sys.stdout = filepath_2
        outfile.write(my_json_out_dict)
    sys.stdout = sys.__stdout__
    
    

Writing create_json.py


In [1]:
import links_list, logo_df_generator
link_list, vid_list = links_list.lists()
video_uri = link_list[12]

p,q,r=logo_df_generator.predict_logo(video_uri)

Waiting for operation to complete...
Entity Id : /m/07rnx
Description : Telstra

	Start Time Offset : 12.480000000
	End Time Offset : 14.880000000
	Confidence : 0.9451204538345337
	Average area covered: 0.3184%
	Max appeared logo position: Lower right 
----------------------------------	frames: 21 ----------------------------------

		Left : 0.9461739659309387
		Top : 0.8956134915351868
		Right : 0.9862479567527771
		Bottom : 0.9706196188926697
		Percentage of screen covered :0.301%
		Logo position :Lower right

		Left : 0.9460610151290894
		Top : 0.8952292799949646
		Right : 0.9861425161361694
		Bottom : 0.970008909702301
		Percentage of screen covered :0.3%
		Logo position :Lower right

		Left : 0.9455258250236511
		Top : 0.894978404045105
		Right : 0.9861322641372681
		Bottom : 0.9705711007118225
		Percentage of screen covered :0.307%
		Logo position :Lower right

		Left : 0.9455810189247131
		Top : 0.8948510885238647
		Right : 0.9860286712646484
		Bottom : 0.970456600189209
		Perce

Processing video "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files/Australian Balet Show - Sylvia 15 TVC.mp4"...
------------------------------ Detected logos: 4 -------------------------------
 85% |  12.360 |  14.880 |  22 fr. | /m/080gwcl      | Council for Chemical Research
 95% |  12.480 |  14.880 |  21 fr. | /m/07rnx        | Telstra
 91% |   1.800 |   6.720 |  42 fr. | /m/06cknc       | The Australian Ballet
 91% |   6.840 |  10.320 |  30 fr. | /m/06cknc       | The Australian Ballet
 91% |  10.440 |  12.120 |  15 fr. | /m/06cknc       | The Australian Ballet
 86% |  12.480 |  14.880 |  21 fr. | /m/06cknc       | The Australian Ballet
 92% |  12.360 |  14.880 |  22 fr. | /m/0bs8cg2      | Bloch
Logo with max frames: The Australian Ballet
Total frames with ' The Australian Ballet ' logo: 108


## Transcription using Vosk

In [49]:
#!pip3 install vosk

In [50]:
# git clone https://github.com/alphacep/vosk-api

In [51]:
# wget https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip

In [52]:
%%writefile Word.py
class Word:
    ''' A class representing a word from the JSON format for vosk speech recognition API '''

    def __init__(self, dict):
        '''
        Parameters:
          dict (dict) dictionary from JSON, containing:
            conf (float): degree of confidence, from 0 to 1
            end (float): end time of the pronouncing the word, in seconds
            start (float): start time of the pronouncing the word, in seconds
            word (str): recognized word
        '''

        self.conf = dict["conf"]
        self.end = dict["end"]
        self.start = dict["start"]
        self.word = dict["word"]
#         return dict
    def to_string(self):
        ''' Returns a string describing this instance '''
#         return self.word, self.start, self.end, self.conf*100
        return "{:20} from {:.2f} sec to {:.2f} sec, confidence is {:.2f}%".format(
            self.word, self.start, self.end, self.conf*100)
    def to_dict(self):
        ''' Returns a values describing this instance '''
        return self.word, self.start, self.end, self.conf
#         return "{:20} from {:.2f} sec to {:.2f} sec, confidence is {:.2f}%".format(
#             self.word, self.start, self.end, self.conf*100)

Overwriting Word.py


In [16]:
%%writefile vosk_transcription.py
def vosk_transcription(video_link):
    import wave
    import json

    from vosk import Model, KaldiRecognizer, SetLogLevel
    import Word as custom_Word
    #convert video to audio
    import moviepy.editor as mp
    vosk_audio_out_link = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/vosk/"+n+".wav"
    # Insert Local Video File Path
    clip = mp.VideoFileClip(video_link)

    # Insert Local Audio File Path
    clip.audio.write_audiofile( vosk_audio_out_link)
    model_path ="vosk-model-en-us-0.22"
    audio_filename = vosk_audio_out_link#"C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/Apple iWatch Series 5 30 TVC.mp4.wav"

    model = Model("vosk-model-en-us-0.22")
    wf = wave.open(audio_filename, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)

    # get the list of JSON dictionaries
    results = []
    # recognize speech using vosk model
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            part_result = json.loads(rec.Result())
            results.append(part_result)
    part_result = json.loads(rec.FinalResult())
    results.append(part_result)

    # convert list of JSON dictionaries to list of 'Word' objects
    list_of_Words = []
    for sentence in results:
        if len(sentence) == 1:
            # sometimes there are bugs in recognition 
            # and it returns an empty dictionary
            # {'text': ''}
            continue
        for obj in sentence['result']:
            w = custom_Word.Word(obj)  # create custom Word object
            list_of_Words.append(w)  # and add it to list

    wf.close()  # close audiofile

    # output to the screen
    for word in list_of_Words:
        print(word.to_dict())

Writing vosk_transcription.py


In [54]:
#vosk_transcription(video_link)

In [55]:
# import sys
# file = open('output.txt', 'a')
# sys.stdout = file

# maps_entity_id = "/m/03s14w"
# print_logo_frames(results, maps_entity_id)

# file.close()

In [56]:
# import sys
# maps_entity_id = "/m/03s14w"
# print_logo_frames(results, maps_entity_id)

# sys.stdout = open("/home/scilab/Desktop/test.txt", "a")
# sys.stdout.close()

### Function for printing Label and Object outputs

In [57]:
import sys
def label_and_object(video):

    results = detect_shot_changes(video)
    shots = print_video_shots(results)

    lab = detect_labels(video, mode, [segment])
    video_labels=print_video_labels(lab)
    shot_labels=print_shot_labels(lab)
    objects = track_objects(video, [segment])
    detected_objects = print_detected_objects(objects)
#     f = open(filename, "w")
#     f.write("Shot segments \n")
#     f.write(shots)
#     print("Video level annotation")
#     print(video_labels, file=f)
#     print("Video level annotation")
#     print(video_labels, file=f)
#     print("Shot level annotation")
#     print(shot_labels, file=f)
#     print("Object level annotation")
#     print(detected_objects, file=f)

    
    


In [58]:
def create_textoutput_label_object(video):
    original_stdout = sys.stdout # Save a reference to the original standard output
    for name in vid_list:
        if name in video:
            f = str(name)
    filename = "C:/Users/manas_nmr2rze/Desktop/Capstone/Output/"+f+".txt" 
    
    with open(filename, 'w') as f:
        sys.stdout = f # Change the standard output to the file we created.
        label_and_object(video)
        sys.stdout = original_stdout 
    

### Function for printing LOGO detection outputs

In [59]:
import sys
def logo_and_frames(video):
    detect_logos(video)

    results = detect_logos(video, [segment])
    entity = print_detected_logos(results)
    detect_logo(local_file_path=video)
    print(detect_logos(video))


In [60]:
def create_textoutput_logo_and_frames(video):
    original_stdout = sys.stdout # Save a reference to the original standard output
    for name in vid_list:
        if name in video:
            f = str(name)
    filename = "C:/Users/manas_nmr2rze/Desktop/Capstone/Output/logo/"+f+".txt" 
    
    with open(filename, 'w') as f:
        sys.stdout = f # Change the standard output to the file we created.
        logo_and_frames(video)
        sys.stdout = original_stdout 

### Print Label and Object outputs in text files

In [61]:
# for i in link_list[:1]:
#     create_textoutput_label_object(i)

### Print LOGO detection outputs in text files

In [62]:
# for j in link_list[35:56]:
#     create_textoutput_logo_and_frames(j)

### Multiple Label detection

In [63]:
# from google.cloud import videointelligence_v1 as vi

# with io.open(video_uri,'rb') as movie:
#         input_content = movie.read()
# video_client = vi.VideoIntelligenceServiceClient()
# video_uri = video_link
# features = [
#     vi.Feature.SHOT_CHANGE_DETECTION,
#     vi.Feature.LABEL_DETECTION,
#     vi.Feature.EXPLICIT_CONTENT_DETECTION,
#     vi.Feature.SPEECH_TRANSCRIPTION,
#     vi.Feature.TEXT_DETECTION,
#     vi.Feature.OBJECT_TRACKING,
#     vi.Feature.LOGO_RECOGNITION,
# ]
# context = vi.VideoContext(
#     segments=...,
#     shot_change_detection_config=...,
#     label_detection_config=...,
#     explicit_content_detection_config=...,
#     speech_transcription_config=...,
#     text_detection_config=...,
#     object_tracking_config=...,
# )
# # operation = video_client.annotate_video(
# #         request={"features": features, "input_content": input_content, "video_context":context}
# #     )
# request = vi.AnnotateVideoRequest(
#     input_uri=video_uri,
#     features=features,
#     video_context=context,
# )
# video_client.annotate_video(request)

## Transcription of videos

In [64]:
#pip install openai

In [65]:
import openai

In [66]:
with open("C:/Users/manas_nmr2rze/Desktop/Capstone/openai/key.txt") as f:
    contents = f.readlines()
for i in contents:
    openai.api_key = i

In [67]:
#!pip install git+https://github.com/huggingface/transformers -q

In [68]:
#!python -m pip install git+https://github.com/huggingface/transformers

In [69]:
#conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [70]:
#pip install pytorch-transformers

In [71]:
#pip install --upgrade pip

In [72]:
#pip install tensorflow

In [73]:

from transformers import pipeline

In [74]:
#converter = pipeline('automatic-speech-recognition', model='openai/whisper-medium')

In [75]:
# import pickle
# pickle.dump(converter, open('speech_rec_converter.pkl','wb'))

In [76]:
import pickle
speech_rec_converter = pickle.load(open('speech_rec_converter.pkl','rb'))

In [77]:
from IPython.display import Audio, display


In [78]:
display(Audio("AdelaideTourism15TVC.mp4"))

In [1]:
#from skimage import io
import io
import os
import pickle
#define the path of folder containing videos
path = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files" 
#assign the object names to a list
file_list=os.listdir(path)

In [2]:
# import re
# vid_list = []
# for string in file_list:
#     m = re.search("^.*mp4$", string)
#     if m:
#         vid_list.append(string)

In [3]:
# link_list=[]
# for name in vid_list:
#     name = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files/"+ name
#     link_list.append(name)

In [4]:
# pickle.dump(vid_list, open('vid_list.pkl','wb'))
# pickle.dump(link_list, open('link_list.pkl','wb'))

In [5]:
# links = pickle.load(open('link_list.pkl','rb'))
# text_converted = {}
# for vid in links:
#     text = speech_rec_converter(vid)
#     text_converted[vid] = text

In [6]:
# pickle.dump(text_converted, open('text_converted.pkl','wb'))

In [7]:
link_list = pickle.load(open('link_list.pkl','rb'))
video_link = (link_list[12])

In [8]:
display(Audio(video_link))

NameError: name 'Audio' is not defined

In [ ]:
#pip install noisereduce


In [9]:
import pickle
import openai
texts = pickle.load(open('text_converted.pkl','rb'))
vid_list = pickle.load(open('vid_list.pkl','rb'))
text_list=[]
i=0
text_dir = {}
for key, value in texts.items():
    for k,v in value.items():
        text_dir[vid_list[i]]= v
        i+=1
 


In [89]:

def key_phrase(video_link):
    video_uri = video_link
    text_results = detect_text(video_link, segments=[segment])
    gen_text = video_text(text_results)
    ph ='' 
    for i in gen_text:
        ph = ph+i+' '
    return ph


ph = key_phrase(video_link)

Processing video "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files/Australia Post - 1 Dollar Coin Collectables 15 TVC.mp4"...
-------------------------------- Detected Text ---------------------------------
Start collecting today.
#Great AussieCoinHunt
Available at participating Post Offices.
Hurry in-store.
Australia Post


### OpenAI Whisper Transcription

In [90]:
#pip install -U openai-whisper

In [91]:
#pip install git+https://github.com/openai/whisper.git 

In [92]:
#To update version
#pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

In [93]:
#pip install setuptools-rust

In [94]:
# import whisper

# model = whisper.load_model("base")
# result = model.transcribe("audio.mp3")
# print(result["text"])

## Transcription of Local files using Google Speech

In [95]:
#conda install -c conda-forge librosa

In [96]:
#pip install librosa

In [ ]:
#conda update --force conda


In [ ]:
#conda update anaconda

In [ ]:
#conda update conda

In [ ]:
#pip install soundfile

In [ ]:
#conda install -c conda-forge pysoundfile

In [ ]:
#pip install ffmpeg

In [ ]:
import os
#import speech_recognition as sr
import ffmpeg
#import soundfile
import numpy as np
#import librosa
import glob

import matplotlib.pyplot as plt
#import librosa.display
from IPython.display import Audio, display

In [ ]:
# import os
# import pickle
# #define the path of folder containing videos
# path = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files" 
# #assign the object names to a list
# file_list=os.listdir(path)
# import re
# vid_list = []
# for string in file_list:
#     m = re.search("^.*mp4$", string)
#     if m:
#         vid_list.append(string)
# link_list=[]
# for name in vid_list:
#     name = "C:/Users/manas_nmr2rze/Desktop/Capstone/NI Ad Feature Extraction Project Files/"+ name
#     link_list.append(name)

In [ ]:
import pickle
link_list = pickle.load(open('link_list.pkl','rb'))
vid_list = pickle.load(open('vid_list.pkl','rb'))
video_link = (link_list[12])

In [ ]:
vid_link = video_link

for name in vid_list:
    if name in vid_link:
        n=name
audio_out_link = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/"+n+".wav"
noise_reduced_audio_out_link = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/rednoise/"+n+".wav"
print(audio_out_link)

In [ ]:
#pip install moviepy

In [ ]:
#convert video to audio
import moviepy.editor as mp
# Insert Local Video File Path
clip = mp.VideoFileClip(video_link)

# Insert Local Audio File Path
clip.audio.write_audiofile(audio_out_link)

In [ ]:
display(Audio(audio_out_link))

In [ ]:
#pip install google-cloud-speech

In [ ]:
#conda install -c numba numba

In [ ]:
#pip install pydub

In [ ]:
#conda install -c conda-forge librosa

In [ ]:
#pip install noisereduce

In [ ]:
#conda install -c fastchan libsndfile

In [ ]:
#conda install -c conda-forge m2w64-libsndfile

In [ ]:
from pydub import AudioSegment
sound = AudioSegment.from_wav(audio_out_link)
sound = sound.set_channels(1)
sound.export(audio_out_link, format="wav")

In [ ]:
# from scipy.io import wavfile
# import noisereduce as nr
# import io
# # load data
# rate, data = wavfile.read(audio_out_link)
# # perform noise reduction
# reduced_noise = nr.reduce_noise(y=data, sr=rate)
# wavfile.write(noise_reduced_audio_out_link, rate, reduced_noise)

In [ ]:
from scipy.io import wavfile

# Load audio file
audio_file = audio_out_link
sample_rate, signal = wavfile.read(audio_file)
import matplotlib.pyplot as plt

# Plot audio signal
plt.figure(figsize=(14, 5))
plt.plot(signal)
plt.title("Audio Signal")
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")
plt.show()
from scipy.signal import wiener

# Perform noise reduction
reduced_noise = wiener(signal, mysize=15)

# Plot noise-reduced signal
plt.figure(figsize=(14, 5))
plt.plot(reduced_noise)
plt.title("Noise-Reduced Signal")
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")
plt.show()
# Save noise-reduced audio file
wavfile.write(noise_reduced_audio_out_link, sample_rate, reduced_noise)


In [ ]:
#pip install google-cloud

In [ ]:
#pip install cachetools

In [ ]:
#pip install google-cloud-speech

In [ ]:
#pip install SpeechRecognition

In [ ]:
from google.cloud import speech
import io
client = speech.SpeechClient()
path = audio_out_link
# path = 'resources/commercial_mono.wav'
with io.open(path, "rb") as audio_file:
    content = audio_file.read()

audio = speech.RecognitionAudio(content=content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,
    language_code="en-US",
    # Enable automatic punctuation
    #enable_automatic_punctuation=True,
    enable_word_time_offsets=True,
    
)

response = client.recognize(config=config, audio=audio)
for result in response.results:
    alternative = result.alternatives[0]
    print("Transcript: {}".format(alternative.transcript))
    print("Confidence: {}".format(alternative.confidence))



In [ ]:
import speech_recognition as sr

# Create a Recognizer instance
r = sr.Recognizer()

# Load the audio file
with sr.AudioFile(audio_out_link) as source:
    audio = r.record(source)  # read the entire audio file

# Transcribe the audio file
text = r.recognize_google(audio)
print(text)
# Print the transcribed text
print('Transcript: {}'.format(text))


## Transcription using ASSEMBLY AI

In [ ]:
#!pip install assemblyai


In [ ]:
#pip install requests

In [ ]:
#!git clone https://github.com/AssemblyAI-Examples/python-speech-recognition.git

In [4]:
%%writefile aai_transcription.py
def aai_audio_transcription(audio_out_link):
    import requests
    import time
    UPLOAD_ENDPOINT = "https://api.assemblyai.com/v2/upload"
    TRANSCRIPTION_ENDPOINT = "https://api.assemblyai.com/v2/transcript"
    api_key = "1c9d460c0979424d8602d6b37dabcca8"
    headers = {"authorization": api_key, "content-type": "application/json"}
    def read_file(filename):
       with open(filename, 'rb') as _file:
           while True:
               data = _file.read(5242880)
               if not data:
                   break
               yield data
    upload_response = requests.post(UPLOAD_ENDPOINT, headers=headers, data=read_file(audio_out_link))
    audio_url = upload_response.json()["upload_url"]
    transcript_request = {'audio_url': audio_url}
    transcript_response = requests.post(TRANSCRIPTION_ENDPOINT, json=transcript_request, headers=headers)
    _id = transcript_response.json()["id"]
    # _confidence = transcript_response.json()["confidence"]
    # _audio_duration = transcript_response.json()["audio_duration"]
    while True:

        polling_response = requests.get(TRANSCRIPTION_ENDPOINT+"/"+_id, headers=headers)

        if polling_response.json()['status'] == 'completed':


            aai_word_text = (polling_response.json()['text'])
            audio_dict = (polling_response.json()['words'])
            keyphrase = (polling_response.json()['results.text'])
            keyphrase_rank = (polling_response.json()['results.rank'])
            keyphrase_timestamp = (polling_response.json()['results.timestamp'])
            break






    #         with open(f"{_id}.txt", 'w') as f:



    #             #f.write(polling_response.json()['text'])
    #             #f.write(str(polling_response.json()['confidence']))
    #             f.write(str(polling_response.json()['words']))

    #             print(f"Transcript saved to {_id}.txt")
    #             break
        elif polling_response.json()['status'] == 'error':


            raise Exception("Transcription failed. Make sure a valid API key has been used.")
        else:
            print("Transcription queued or processing ...")
        time.sleep(5)
    aai_word_dict = {}
    for i in range(len(audio_dict)):
    
        aai_word_dict[audio_dict[i]['text']]=(audio_dict[i]['start']/1000,
                                              audio_dict[i]['end']/1000,
                                              audio_dict[i]['confidence'])
    return aai_word_text, aai_word_dict, keyphrase, keyphrase_rank, keyphrase_timestamp

Overwriting aai_transcription.py


In [ ]:
audio_text, audio_dict, keyphrase, keyphrase_rank, keyphrase_timestamp = aai_audio_transcription(audio_out_link)

In [ ]:
print(audio_text)

In [ ]:
print(audio_dict)

In [ ]:
#audio_dict[i]['start']*1000

In [ ]:
display(Audio(audio_out_link))

In [18]:
%%writefile google_local_transcription.py
def local_google_transcribe(vid_link):
    
    import os
    import speech_recognition as sr
    import ffmpeg
    #import soundfile
    import numpy as np
    #import librosa
    import glob

    import matplotlib.pyplot as plt
    #import librosa.display
    from IPython.display import Audio, display
    from google.cloud import speech
    import io
    for name in vid_list:
        if name in vid_link:
            n=name
    audio_out_link = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/"+"output"+".wav"
    from pydub import AudioSegment
    #convert video to audio
    import moviepy.editor as mp
    # Insert Local Video File Path
    clip = mp.VideoFileClip(vid_link)

    # Insert Local Audio File Path
    clip.audio.write_audiofile(audio_out_link)
    sound = AudioSegment.from_wav(audio_out_link)
    sound = sound.set_channels(1)
    sound.export(audio_out_link, format="wav")
    words =[]
    word_dict = {}
    word_str = ''

    client = speech.SpeechClient()
    #path = audio_out_link
    # path = 'resources/commercial_mono.wav'
    with io.open(audio_out_link, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code="en-US",
        # Enable automatic punctuation
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True,

    )
#     print(f"start_time      end_time        Word   ")
#     print("=======================================")
    response = client.recognize(config=config, audio=audio)
    for result in response.results:
        alternative = result.alternatives[0]
    #     print("Transcript: {}".format(alternative.transcript))
    #     print("Confidence: {}".format(alternative.confidence))


        for word_info in alternative.words:
            word = word_info.word
            word = word.lower()
            start_time = word_info.start_time
            end_time = word_info.end_time
            words.append(word)
            word_str= word_str+' '+word
            word_dict[word]=(start_time.total_seconds(), end_time.total_seconds())
#             print(f"{start_time.total_seconds():.2f}      \t{end_time.total_seconds():.2f}    \t{word}")
    return words, word_dict, word_str

Writing google_local_transcription.py


In [ ]:
words, word_dict,audio_text=local_google_transcribe(video_link)

In [ ]:
word_dict

In [ ]:
#pip install openai

### Transcription using ravai

In [ ]:
#pip install rev-ai

In [14]:
%%writefile ravai_transcription.py
def ravai_transcribe(alink):
    from rev_ai import apiclient

    token = "02hFR4d_rGZ7Bz5ckG1wrYIbwHuF8FxTig6b3OjIP3G6rUA6puNyBo4CuekXULZgtBgH1fqe1J6Vq9QhfRTVKEVi4wGP4"
    filePath = alink

    # create your client
    client = apiclient.RevAiAPIClient(token)

    # send a local file
    job = client.submit_job_local_file(filePath)
    text_stream = client.get_transcript_text_as_stream(job.id)
    # check job status
#     job_details = client.get_job_details(job.id)
    
#     # retrieve transcript as text
#     transcript_text = client.get_transcript_text(job.id)

#     # retrieve transcript as JSON
#     transcript_json = client.get_transcript_json(job.id)

#     # retrieve transcript as a Python object
#     transcript_object = client.get_transcript_object(job.id)
    
    return text_stream #transcript_text, transcript_json, transcript_object

Overwriting ravai_transcription.py


In [4]:
 from rev_ai import apiclient

token = "02hFR4d_rGZ7Bz5ckG1wrYIbwHuF8FxTig6b3OjIP3G6rUA6puNyBo4CuekXULZgtBgH1fqe1J6Vq9QhfRTVKEVi4wGP4"
filePath = audio_link

# create your client
client = apiclient.RevAiAPIClient(token)

# send a local file
job = client.submit_job_local_file(filePath)

# check job status
# job_details = client.get_job_details(job.id)

# # retrieve transcript as text
# transcript_text = client.get_transcript_text(job.id)

# # retrieve transcript as JSON
# transcript_json = client.get_transcript_json(job.id)

# # retrieve transcript as a Python object
# transcript_object = client.get_transcript_object(job.id)


In [7]:
job_details = client.get_job_details(job.id)


In [11]:
transcript_text = client.get_transcript_text(job.id)

# retrieve transcript as JSON
transcript_json = client.get_transcript_json(job.id)

# retrieve transcript as a Python object
transcript_object = client.get_transcript_object(job.id)
transcript_text
transcript_object
transcript_json

{'monologues': [{'speaker': 0,
   'elements': [{'type': 'text',
     'value': "We're",
     'ts': 0.35,
     'end_ts': 0.83,
     'confidence': 0.6},
    {'type': 'punct', 'value': ' '},
    {'type': 'text',
     'value': 'sending',
     'ts': 0.83,
     'end_ts': 1.07,
     'confidence': 0.68},
    {'type': 'punct', 'value': ' '},
    {'type': 'text',
     'value': 'you',
     'ts': 1.07,
     'end_ts': 1.19,
     'confidence': 0.63},
    {'type': 'punct', 'value': ' '},
    {'type': 'text',
     'value': 'to',
     'ts': 1.19,
     'end_ts': 1.43,
     'confidence': 0.67},
    {'type': 'punct', 'value': ' '},
    {'type': 'text',
     'value': 'profile',
     'ts': 1.43,
     'end_ts': 2.07,
     'confidence': 0.94},
    {'type': 'punct', 'value': ' '},
    {'type': 'text',
     'value': 'on',
     'ts': 2.07,
     'end_ts': 2.19,
     'confidence': 0.95},
    {'type': 'punct', 'value': ' '},
    {'type': 'text',
     'value': 'the',
     'ts': 2.19,
     'end_ts': 2.55,
     'confid

In [13]:
from rev_ai.models import MediaConfig
from rev_ai.streamingclient import RevAiStreamingClient
import io


"""
Name of file to be transcribed
"""
filename = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/new_audio_out1/AAMI Insurance 30 TVC.wav"

"""
String of your access token
"""
access_token = "02hFR4d_rGZ7Bz5ckG1wrYIbwHuF8FxTig6b3OjIP3G6rUA6puNyBo4CuekXULZgtBgH1fqe1J6Vq9QhfRTVKEVi4wGP4"

"""
Media configuration of audio file.
This includes the content type, layout, rate, format, and # of channels
"""
config = MediaConfig("audio/x-raw", "interleaved", 16000, "S16LE", 1)

"""
Create client with your access token and media configuration
"""
streamclient = RevAiStreamingClient(access_token, config)

"""
Open file and read data into array.
Practically, stream data would be divided into chunks
"""
with io.open(filename, 'rb') as stream:
    MEDIA_GENERATOR = [stream.read()]

"""
Starts the streaming connection and creates a thread to send bytes from the
MEDIA_GENERATOR. response_generator is a generator yielding responses from
the server
"""
response_generator = streamclient.start(MEDIA_GENERATOR)

"""
Iterates through the responses from the server when obtained
"""
for response in response_generator:
    print(response)

"""
Ends the connection early. Not needed as the server will close the connection
upon receiving an "EOS" message.
"""
streamclient.end()
print(response_generator)

Connected, Job ID : fNhqX5eEUvKb5Vhp
Connection Closed. Code : 1000; Reason : End of input. Closing
<generator object RevAiStreamingClient._get_response_generator at 0x0000024F292D5540>


In [2]:
audio_link_list, audio_list = audio_links.lists()
audio_link = (str(audio_link_list[13]))
audio_link

NameError: name 'audio_links' is not defined

In [3]:
import ravai_transcription
import audio_links
audio_link_list, audio_list = audio_links.lists()
audio_link = (str(audio_link_list[10]))
transcript_text, transcript_json, transcript_object = ravai_transcription.ravai_transcribe(audio_link)

HTTPError: 409 Client Error: Conflict for url: https://api.rev.ai/speechtotext/v1/jobs/dZ3HCxKtO3vxArdt/transcript; Server Response : {"allowed_values":["transcribed"],"current_value":"in_progress","type":"https://www.rev.ai/api/v1/errors/invalid-job-state","title":"Job is in invalid state","status":409,"detail":"Job is in invalid state to obtain the output","extensions":{}}

# KeyPhrase Extraction

### USing OpenAI API

In [ ]:
import openai
for name in vid_list:
    if name in video_link:
        n=name

phrase = audio_text+ph
# def generate_code(prompt):
#     completions = openai.Completion.create(
#         engine="davinci-codex",
#         prompt=prompt,
#         max_tokens=1024,
#         n=1,
#         stop=None,
#         temperature=0.5,
#     )
# def openai_keyphrase(phrase):
    
#     response = openai.Completion.create(
#       model="text-davinci-003",
#       prompt=phrase,
#       temperature=.5,
#       max_tokens=20,
#       top_p=1,
#       frequency_penalty=.8,
#       presence_penalty=0
#     return 
    
# )
print(phrase)

## Generate text corpus

In [ ]:
video_link = (str(link_list[12]))


In [12]:
%%writefile text_corpus.py
def google_text_corpus(video_link):
    words, word_dict,audio_text=local_google_transcribe(video_link)
#     print(audio_text)
    ph = key_phrase(video_link)
#     print(ph)
    phrase = audio_text+ph
    return phrase
def openai_text_corpus(video_link):
    import pickle
    speech_rec_converter = pickle.load(open('speech_rec_converter.pkl','rb'))
    aud_text = speech_rec_converter(video_link)
   
#     print(aud_text['text'])
    ph = key_phrase(video_link)
#     print(ph)
    phrase = aud_text['text']+' '+ph
    return phrase
def assemblyai_text_corpus(video_link):
    import moviepy.editor as mp
    # Insert Local Video File Path
    audio_out_link = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/"+'output'+".wav"
    clip = mp.VideoFileClip(video_link)

    # Insert Local Audio File Path
    clip.audio.write_audiofile(audio_out_link)
    audio_text, audio_dict = aai_audio_transcription(audio_out_link)

   
#     print(aud_text['text'])
    ph = key_phrase(video_link)
#     print(ph)
    phrase = audio_text+' '+ph
    return phrase, audio_dict,keyphrase_rank,keyphrase_timestamp
def aai_audio_transcription(audio_out_link):
    import requests
    import time
    import string
    UPLOAD_ENDPOINT = "https://api.assemblyai.com/v2/upload"
    TRANSCRIPTION_ENDPOINT = "https://api.assemblyai.com/v2/transcript"
    api_key = "1c9d460c0979424d8602d6b37dabcca8"
    headers = {"authorization": api_key, "content-type": "application/json","auto_highlights": 'True',"iab_categories": 'True'}
    def read_file(filename):
       with open(filename, 'rb') as _file:
           while True:
               data = _file.read(5242880)
               if not data:
                   break
               yield data
    upload_response = requests.post(UPLOAD_ENDPOINT, headers=headers, data=read_file(audio_out_link))
    audio_url = upload_response.json()["upload_url"]
    transcript_request = {'audio_url': audio_url}
    transcript_response = requests.post(TRANSCRIPTION_ENDPOINT, json=transcript_request, headers=headers)
    _id = transcript_response.json()["id"]
    # _confidence = transcript_response.json()["confidence"]
    # _audio_duration = transcript_response.json()["audio_duration"]
    while True:

        polling_response = requests.get(TRANSCRIPTION_ENDPOINT+"/"+_id, headers=headers)

        if polling_response.json()['status'] == 'completed':


            aai_word_text = str((polling_response.json()['text']))
            audio_dict = (polling_response.json()['words'])

            break






    #         with open(f"{_id}.txt", 'w') as f:



    #             #f.write(polling_response.json()['text'])
    #             #f.write(str(polling_response.json()['confidence']))
    #             f.write(str(polling_response.json()['words']))

    #             print(f"Transcript saved to {_id}.txt")
    #             break
        elif polling_response.json()['status'] == 'error':


            raise Exception("Transcription failed. Make sure a valid API key has been used.")
        else:
            print("Transcription queued or processing ...")
        time.sleep(5)
    aai_word_dict = {}
    for i in range(len(audio_dict)):
        removePuncuation= audio_dict[i]['text'].translate(str.maketrans('', '', string.punctuation)).lower()
        
        
    
        aai_word_dict[removePuncuation]=(audio_dict[i]['start']/1000,
                                              audio_dict[i]['end']/1000,
                                              audio_dict[i]['confidence'])
    return aai_word_text, aai_word_dict

Overwriting text_corpus.py


In [2]:
%%writefile aai_text_corpus.py
def assemblyai_text_corpus(video_link,gen_text):# get_text is gen_text = video_text(text_results)
    import moviepy.editor as mp
    
    import string
    # Insert Local Video File Path
    audio_out_link = "C:/Users/manas_nmr2rze/Desktop/Capstone/AudioOutput/"+'output'+".wav"
    clip = mp.VideoFileClip(video_link)

    # Insert Local Audio File Path
    clip.audio.write_audiofile(audio_out_link)
   
    audio_text, audio_dict, aai_time= aai_audio_transcription(audio_out_link)
    string = ''
    for word in gen_text:
        string=string+' '+word
        

    text = string.split(' ')
    vid_text ='' 
    for i in text:
        vid_text = vid_text+i+' ' 
    vid_text = str(vid_text).lower()

    phrase = audio_text+' '+ vid_text

    return phrase, audio_dict, audio_text, vid_text, aai_time

def aai_audio_transcription(audio_out_link):
    import requests
    import time
    import string
    UPLOAD_ENDPOINT = "https://api.assemblyai.com/v2/upload"
    TRANSCRIPTION_ENDPOINT = "https://api.assemblyai.com/v2/transcript"
    api_key = "1c9d460c0979424d8602d6b37dabcca8"
    headers = {"authorization": api_key, "content-type": "application/json","auto_highlights": 'True',"iab_categories": 'True'}
    def read_file(filename):
       with open(filename, 'rb') as _file:
           while True:
               data = _file.read(5242880)
               if not data:
                   break
               yield data
    upload_response = requests.post(UPLOAD_ENDPOINT, headers=headers, data=read_file(audio_out_link))
    audio_url = upload_response.json()["upload_url"]
    transcript_request = {'audio_url': audio_url}
    transcript_response = requests.post(TRANSCRIPTION_ENDPOINT, json=transcript_request, headers=headers)
    _id = transcript_response.json()["id"]
    # _confidence = transcript_response.json()["confidence"]
    # _audio_duration = transcript_response.json()["audio_duration"]
    while True:

        polling_response = requests.get(TRANSCRIPTION_ENDPOINT+"/"+_id, headers=headers)

        if polling_response.json()['status'] == 'completed':


            aai_word_text = str((polling_response.json()['text']))
            audio_dict = (polling_response.json()['words'])

            break



    #         with open(f"{_id}.txt", 'w') as f:



    #             #f.write(polling_response.json()['text'])
    #             #f.write(str(polling_response.json()['confidence']))
    #             f.write(str(polling_response.json()['words']))

    #             print(f"Transcript saved to {_id}.txt")
    #             break
        elif polling_response.json()['status'] == 'error':


            raise Exception("Transcription failed. Make sure a valid API key has been used.")
        else:
            print("Transcription queued or processing ...")
        time.sleep(5)
    aai_word_dict = {}
    aai_time_list = []
    for i in range(len(audio_dict)):
        removePuncuation= audio_dict[i]['text'].translate(str.maketrans('', '', string.punctuation)).lower()
        
        
    
        aai_word_dict[removePuncuation]=(audio_dict[i]['start']/1000,
                                              audio_dict[i]['end']/1000,
                                              audio_dict[i]['confidence'])
        aai_time_list.append((audio_dict[i]['start']/1000,
                                              audio_dict[i]['end']/1000,
                                              audio_dict[i]['confidence']))
    return aai_word_text, aai_word_dict, aai_time_list

def generate_audiofiles(audio_folder_path,links_list):
    
    import moviepy.editor as mp
    import string
    
    link_list, vid_list = links_list.lists()
    for name in vid_list:
        try:


            for link in link_list:
                if name in link:
                    video = name
                    vlink = link
        except:
            continue
        try:

            # Insert Local Video File Path
            name=video.replace('.mp4','')
            audio_out_link = audio_folder_path+name+".wav"
            clip = mp.VideoFileClip(vlink)

            # Insert Local Audio File Path
            clip.audio.write_audiofile(audio_out_link)
        except:
            continue


Overwriting aai_text_corpus.py


In [16]:
import string

test_str = 'Gfg, is best: for !{./} Geeks ;'

test_str = test_str.translate(str.maketrans('', '', string.punctuation))
print(test_str)


Gfg is best for  Geeks 


In [17]:
google_text_corpus(video_link)

NameError: name 'google_text_corpus' is not defined

In [ ]:
# import pickle
# speech_rec_converter = pickle.load(open('speech_rec_converter.pkl','rb'))
# text = speech_rec_converter(video_link)

In [ ]:
assemblyai_text_corpus(video_link)

In [ ]:
phrase = assemblyai_text_corpus(video_link)[0]

In [ ]:
display(Audio(video_link))

In [ ]:
print(phrase)

In [4]:
%%writefile generate_keyphrase_branding.py
import sys, detect_texts, aai_text_corpus,keyphrase_extraction
def create_keyphrase_sentiment_branding(video):
    import sys, detect_texts, aai_text_corpus,keyphrase_extraction
    import logo_df_generator
    import json
    phrase, audio_dict, audio_text, vid_text,time_list = audio_out(video)
    keyphrase=keyphrase_extraction.keyphrase_bart(phrase, audio_dict, audio_text, vid_text,time_list)
    sentiment_scores =keyphrase_extraction.vader_sentiment(phrase)
    desc=logo_df_generator.predict_logo(video)
    print(f"Branding information : ")
    print(desc)
    audio_dict_json = json.dumps(audio_dict)
    audio_text_json= json.dumps({'Audio text':audio_text})
    video_text_json= json.dumps({'Video text':vid_text})
    keyphrase_json = json.dumps({'Keyphrase':keyphrase})
    sentiment_json= json.dumps(sentiment_scores)
    logo_json=json.dumps(desc)
    
    
#     print("Keyphrase of video :")
#     print(keyphrase)
#     print("Sentiment scores of the video :")
#     print(sentiment_scores)
    return keyphrase, sentiment_scores
def audio_out(video):
    import sys, detect_texts, aai_text_corpus,keyphrase_extraction
    try:
        text_results = detect_texts.detect(video)
        vid_text=detect_texts.video_text(text_results)
        phrase, audio_dict, audio_text, vid_text,time_list = aai_text_corpus.assemblyai_text_corpus(video,vid_text)

        return phrase, audio_dict, audio_text, vid_text,time_list

    except:
        pass
    

Writing generate_keyphrase_branding.py


In [88]:
%%writefile keyphrase_extraction.py
def keyphrase_openai(phrase):
    import openai
    with open("C:/Users/manas_nmr2rze/Desktop/Capstone/openai/key.txt") as f:
        
        contents = f.readlines()
        for i in contents:
            openai.api_key = i

    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=phrase,
      temperature=.5,
      max_tokens=20,
      top_p=1,
      frequency_penalty=.8,
      presence_penalty=0
    )
    choice = response['choices']
    key = choice[0].text
    key = key.replace('\n\nKeywords:','')
    key = key.replace('\n','')
    return key

def keyphrase_yake(phrase,word_dict,audio_text, vid_text,time_list):
    from yake import KeywordExtractor as Yake
    yake = Yake(lan="en")
    yake_keyphrases = yake.extract_keywords(phrase)
    #print(yake_keyphrases)
    import yake
    
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(phrase)
    language = "en"
    max_ngram_size = 6
    deduplication_threshold = 0.8
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 1

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(phrase)
    
    
       


    for kw in keywords:
        print(kw[0])
    keyphrase = kw[0]
    keyphr = keyphrase.lower()
    keyph = keyphr.split(' ')
    start, end = substring_pos(phrase, keyphrase)
    key=keyphrase.upper()
    try:
        stime = time_list[start][0]
        etime = time_list[end][1]
         
#         stime = word_dict[keyph[0]][0]
#         etime = word_dict[keyph[-1]][1]
        
        #print("")
        if keyphrase.lower()in audio_text.lower():
            response = str('Keyphrase is in audio....')
        elif keyphrase.lower()in vid_text.lower():
            response = str('Keyphrase is in visual text..')
        return key, stime, etime, response
    except:
        response = str('Keyphrase timestamps not found..')
        return key, response
    #print("=======================")
#     print(f"Keyphrase starting time: {stime} seconds")
#     print(f"Keyphrase ending time: {etime} seconds")
def keyphrase_suma(phrase,word_dict,audio_text, vid_text,time_list):
    from summa import keywords
    TR_keywords = keywords.keywords(phrase, scores=True)
    print(TR_keywords[0:15])

    from keybert import KeyBERT
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    #kw_model = pickle.load(open('kw_model.pkl','rb'))
    keywords = kw_model.extract_keywords(phrase, 

                                         keyphrase_ngram_range=(1, 8), 

                                         stop_words='english', 

                                         highlight=False,

                                         top_n=1);

    keywords_list= list(dict(keywords).keys());

    keyphrase = keywords_list[0].lower();
    keyph = keyphrase.split(' ');
    start, end = substring_pos(phrase, keyphrase)
    key=keyphrase.upper()
    
    try:
        stime = time_list[start][0]
        etime = time_list[end][1]
         

        #print("")
        if keyphrase.lower()in audio_text.lower():
            response = str('Keyphrase is in audio....')
        elif keyphrase.lower()in vid_text.lower():
            response = str('Keyphrase is in visual text..')
        return key, stime, etime, response
    except:
        response = str('Keyphrase timestamps not found..')
        return key, response


def keyphrase_rake1(phrase,word_dict,audio_text, vid_text,time_list):
    from rake_nltk import Rake

    # Initialize the Rake object
    r = Rake()

    # Text to extract key phrases from
    text = phrase
    # Extract keywords
    r.extract_keywords_from_text(text)

    # Get the top 5 phrases
    results = r.get_ranked_phrases()[:3]
    keyph_list = []

    # Print the phrases
    for keyphrase in results:
        try:

            keyphr = keyphrase.lower()
            keyph = keyphr.split(' ')
            start, end = substring_pos(phrase, keyphrase)
            key=keyphrase.upper()

            try:
                stime = time_list[start][0]
                etime = time_list[end][1]


                #print("")
                if keyphrase.lower()in audio_text.lower():
                    response = str('Keyphrase is in audio....')
                elif keyphrase.lower()in vid_text.lower():
                    response = str('Keyphrase is in visual text..')
                keyph_list.append((key, stime, etime, response));
            except:
                response = str('Keyphrase timestamps not found..')
                keyph_list.append((key, response));

#             print(keyphrase.upper())
#             print("=======================")
#             print(f"Keyphrase starting time: {stime} seconds")
#             print(f"Keyphrase ending time: {etime} seconds")
        except:
            pass
    
    print(" Top three Keyphrases ".ljust(80, "-"))
    return keyph_list

def keyphrase_spacy(phrase,word_dict,audio_text, vid_text,time_list):
    import spacy

    # Load the English language model
    nlp = spacy.load('en_core_web_lg')

    # Text to extract keywords from
    text = phrase
    # Process the text with spaCy
    doc = nlp(text)

    # Extract keywords
#     keywords = []
#     for token in doc:
#         if not token.is_stop and token.pos_ in ['NOUN', 'PROPN','ADJECTIVE']:
#             keywords.append(token.text)
    keyphrases = set()
    for chunk in doc.noun_chunks:
        if len(chunk) > 1 and chunk.root.pos_ != 'PRON':
            keyphrases.add(chunk.text)
    keyphrases = list(keyphrases)
    keyphrase = keyphrases[0]
    start, end = substring_pos(phrase, keyphrase)
    key=keyphrase.upper()
    
    try:
        stime = time_list[start][0]
        etime = time_list[end][1]
         

        #print("")
        if keyphrase.lower()in audio_text.lower():
            response = str('Keyphrase is in audio....')
        elif keyphrase.lower()in vid_text.lower():
            response = str('Keyphrase is in visual text..')
        return key, stime, etime, response
    except:
        response = str('Keyphrase timestamps not found..')
        return key, response

#     keyphr = keyphrase.lower()
#     keyph = keyphr.split(' ')
#     try:
#         stime = word_dict[keyph[0]][0]
#         etime = word_dict[keyph[-1]][1]
#         keyphrase=keyphrase.upper()

#         return keyphrase, stime, etime
#     except:
#         return keyphrase
# #     return keyphrases[0]
def keyphrase_bart(phrase,word_dict,audio_text, vid_text,time_list):
#     import torch
    from transformers import BartTokenizer, BartForConditionalGeneration

    # Load the BART model and tokenizer
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

    # Define a sample input text
    input_text = phrase

    # Preprocess the input text
    input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors='pt')

    # Generate keyphrases using BART
    output = model.generate(input_ids, max_length=20, num_beams=4, early_stopping=True)
    keyphrase = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print the generated keyphrases
#     print(keyphrase)

    start, end = substring_pos(phrase, keyphrase)
    key=keyphrase.upper()
    try:
        stime = time_list[start][0]
        etime = time_list[end][1]
         

        #print("")
        if keyphrase.lower()in audio_text.lower():
            response = str('Keyphrase is in audio....')
        elif keyphrase.lower()in vid_text.lower():
            response = str('Keyphrase is in visual text..')
        return key, stime, etime, response
    except:
        response = str('Keyphrase timestamps not found..')
        return key, response
    

   
def vader_sentiment(phrase):
    try:
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        analyzer = SentimentIntensityAnalyzer()
        text = phrase
        scores = analyzer.polarity_scores(text)

        return scores
    except:
        return 'Sentiment analysis not possible'

def substring_pos(string, substring):
    start=0
    end=0
    text = punc_remover(string)
    ph = punc_remover(substring)
    text_l = list(text.split(sep=' '))
    ph_l = list(ph.split(sep=' '))


    for j in range(len(text_l)):
        if ph_l[0]== text_l[j]:
            count=0
            for i in range(len(ph_l)):
                if ph_l[i]== text_l[j+i]:
                    count+=1
            if count== len(ph_l):
                start = j
                end = start+len(ph_l)-1
                break
            else:
                continue
    return start, end
def punc_remover(text):
    import string

    # Remove punctuation using string module and filter function
    text = ''.join(filter(lambda char: char not in string.punctuation, text)).lower()
    return text

#Call to action 
import re

def product_claim(text,word_dict,audio_text, vid_text,time_list):
    import pickle
    import re
    pr_clm_phrases = pickle.load(open('pr_clm_phrases.pkl','rb'))

    # Split the text into sentences using regular expressions
    sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!|\;)\s', text)
    # Iterate over each sentence and check if it contains the phrase
    result = []
    for sentence in sentences:
        for phrase in pr_clm_phrases:
            if phrase.lower() in sentence.lower():
                                
                start, end = substring_pos(text.lower(),sentence.lower())
                key = sentence
                try:
                    stime = time_list[start][0]
                    etime = time_list[end][1]


                    #print("")
                    if phrase.lower()in audio_text.lower():
                        response = str('product claim is in audio....')
                    elif phrase.lower()in vid_text.lower():
                        response = str('product claim is in visual text..')
                    result.append((key, stime, etime, response))
                    
#                     return key, stime, etime, response
                except:
                    response = str('timestamps not found..')
                    result.append((key, response))
                    
    result= list(set(result))                
    if len(result)==0:
        result.append('No product claim sentences found')
        return result
    else:
        header = 'Product claim statement :  '
        return header,result
                 
    

def product_details(text,word_dict,audio_text, vid_text,time_list):
    import pickle
    import re
    pr_dt_phrases = pickle.load(open('pr_dt_phrases.pkl','rb'))
    # Split the text into sentences using regular expressions
    sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!|\;)\s', text)
    # Iterate over each sentence and check if it contains the phrase
    result = []
    for sentence in sentences:
        for phrase in pr_dt_phrases:
            if phrase.lower() in sentence.lower():
                                
                start, end = substring_pos(text.lower(),sentence.lower())
                key = sentence
                try:
                    stime = time_list[start][0]
                    etime = time_list[end][1]


                    #print("")
                    if phrase.lower()in audio_text.lower():
                        response = str('product detail is in audio....')
                    elif phrase.lower()in vid_text.lower():
                        response = str('product detail is in visual text..')
                    result.append((key, stime, etime, response))
                    
#                     return key, stime, etime, response
                except:
                    response = str('timestamps not found..')
                    result.append((key, response))
                    
    result= list(set(result))                
    if len(result)==0:
        result.append('No product detail sentences found')
        return result
    else:
        header = 'Product detail statement :  '
        return header,result
             
    

def product_offers(text,word_dict,audio_text, vid_text,time_list):
    import pickle
    import re
    pr_off_phrases = pickle.load(open('pr_off_phrases.pkl','rb'))
    # Split the text into sentences using regular expressions
    sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!|\;)\s', text)
    # Iterate over each sentence and check if it contains the phrase
    result = []
    for sentence in sentences:
        for phrase in pr_off_phrases:
            if phrase.lower() in sentence.lower():
                                
                start, end = substring_pos(text.lower(),sentence.lower())
                key = sentence
                try:
                    stime = time_list[start][0]
                    etime = time_list[end][1]


                    #print("")
                    if phrase.lower()in audio_text.lower():
                        response = str('product offer is in audio....')
                    elif phrase.lower()in vid_text.lower():
                        response = str('product offer is in visual text..')
                    result.append((key, stime, etime, response))
                    
#                     return key, stime, etime, response
                except:
                    response = str('timestamps not found..')
                    result.append((key, response))
                
    result= list(set(result))                
    if len(result)==0:
        result.append('No product offer sentences found')
        return result
    else:
        header = 'Product offer statement :  '
        return header,result
           
    

def call_to_buy(text,word_dict,audio_text, vid_text,time_list):
    import pickle
    import re
    ctb_phrases = pickle.load(open('ctb_phrases.pkl','rb'))
    # Split the text into sentences using regular expressions
    sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!|\;)\s', text)
    # Iterate over each sentence and check if it contains the phrase
    result = []
    for sentence in sentences:
        for phrase in ctb_phrases:
            if phrase.lower() in sentence.lower():
                                
                start, end = substring_pos(text.lower(),sentence.lower())
                key = sentence
                try:
                    stime = time_list[start][0]
                    etime = time_list[end][1]


                    #print("")
                    if phrase.lower()in audio_text.lower():
                        response = str('call to buy is in audio....')
                    elif phrase.lower()in vid_text.lower():
                        response = str('call to buy is in visual text..')
                    result.append((key, stime, etime, response))
                    
#                     return key, stime, etime, response
                except:
                    response = str('timestamps not found..')
                    result.append((key, response))
                    
                
    result= list(set(result))
    if len(result)==0:
        result.append('No call to buy sentences found')
        return result
    else:
        header = 'Call to buy statement :  '
        return header,result


def call_to_seek_info(text,word_dict,audio_text, vid_text,time_list):
    import pickle
    import re
    ctsmi_phrases = pickle.load(open('ctsmi_phrases.pkl','rb'))
    # Split the text into sentences using regular expressions
    sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!|\;)\s', text)
    # Iterate over each sentence and check if it contains the phrase
    result = []
    for sentence in sentences:
        for phrase in ctsmi_phrases:
            if phrase.lower() in sentence.lower():
                                
                start, end = substring_pos(text.lower(),sentence.lower())
                key = sentence
                try:
                    stime = time_list[start][0]
                    etime = time_list[end][1]


                    #print("")
                    if phrase.lower()in audio_text.lower():
                        response = str('call to seek info is in audio....')
                    elif phrase.lower()in vid_text.lower():
                        response = str('call to seek info is in visual text..')
                    result.append((key, stime, etime, response))
                    
#                     return key, stime, etime, response
                except:
                    response = str('timestamps not found..')
                    result.append((key, response))
                    
                
    result= list(set(result))
    if len(result)==0:
        result.append('No call to seek more info sentences found')
        return result
    else:
        header = 'Call to seek more info statement :  '
        return header,result
def call_to_action(text,word_dict,audio_text, vid_text,time_list):
    import pickle
    import re
    cta_phrases = pickle.load(open('cta_phrases.pkl','rb'))
    # Split the text into sentences using regular expressions
    sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!|\;)\s', text)
    # Iterate over each sentence and check if it contains the phrase
    result = []
    for sentence in sentences:
        for phrase in cta_phrases:
            if phrase.lower() in sentence.lower():
                                
                start, end = substring_pos(text.lower(),sentence.lower())
                key = sentence
                try:
                    stime = time_list[start][0]
                    etime = time_list[end][1]


                    #print("")
                    if phrase.lower()in audio_text.lower():
                        response = str('other call to action is in audio....')
                    elif phrase.lower()in vid_text.lower():
                        response = str('other call to action is in visual text..')
                    result.append((key, stime, etime, response))
                    
#                     return key, stime, etime, response
                except:
                    response = str('timestamps not found..')
                    result.append((key, response))
                    
                
    result= list(set(result))
    if len(result)==0:
        result.append('No other call to action sentences found')
        return result
    else:
        header = 'Other call to action statement :  '
        return header,result


Overwriting keyphrase_extraction.py


In [ ]:
keyphrase_openai(phrase)

In [ ]:
#pip install --upgrade scipy


In [68]:
lst=[]
lst.append(('bkb'))

len(lst)

1

In [87]:
import pandas as pd
import pickle
df = pd.read_excel('marketing_phrases.xlsx')
cta = df['call to action'].dropna().drop_duplicates()
pr_clm = df['product claim'].dropna().drop_duplicates()
pr_dt = df['product details'].dropna().drop_duplicates()
pr_off = df['product offer'].dropna().drop_duplicates()
ctb = df['call to buy'].dropna().drop_duplicates()
ctsmi = df['call to seek info'].dropna().drop_duplicates()
cta_phrases= list(cta)
pr_clm_phrases= list(pr_clm)
pr_dt_phrases= list(pr_dt)
pr_off_phrases= list(pr_off)
ctb_phrases= list(ctb)
ctsmi_phrases= list(ctsmi)
pickle.dump(cta_phrases, open('cta_phrases.pkl','wb'))
pickle.dump(pr_clm_phrases, open('pr_clm_phrases.pkl','wb'))
pickle.dump(pr_dt_phrases, open('pr_dt_phrases.pkl','wb'))
pickle.dump(pr_off_phrases, open('pr_off_phrases.pkl','wb'))
pickle.dump(ctb_phrases, open('ctb_phrases.pkl','wb'))
pickle.dump(ctsmi_phrases, open('ctsmi_phrases.pkl','wb'))
len(ctb_phrases)

690

In [ ]:
from scipy.sparse import coo_matrix


### Using PKE

In [ ]:
# import pke


# # define the set of valid Part-of-Speeches
# pos = {'NOUN', 'PROPN', 'ADJ','VERB'}

# # 1. create a TextRank extractor.
# extractor = pke.unsupervised.TextRank()

# # 2. load the content of the document.
# extractor.load_document(input=phrase,
#                         language='en',
#                         normalization=None)

# # 3. build the graph representation of the document and rank the words.
# #    Keyphrase candidates are composed from the 33-percent
# #    highest-ranked words.
# extractor.candidate_weighting(window=2,
#                               pos=pos,
#                               top_percent=1)

# # 4. get the highest scored candidates as keyphrases
# keyphrases = extractor.get_n_best(n=2)

# keyphrases

In [ ]:
def keyphrase_pke(phrase):
    import pke

    # initialize keyphrase extraction model, here TopicRank
    extractor = pke.unsupervised.TopicRank()

    # load the content of the document, here document is expected to be a simple 
    # test string and preprocessing is carried out using spacy
    extractor.load_document(input=phrase, language='en')

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives (i.e. `(Noun|Adj)*`)
    extractor.candidate_selection()

    # candidate weighting, in the case of TopicRank: using a random walk algorithm
    extractor.candidate_weighting()

    # N-best selection, keyphrases contains the 10 highest scored candidates as
    # (keyphrase, score) tuples
    keyphrases = extractor.get_n_best(n=5)

    print(keyphrases)

In [ ]:
#keyphrase_pke(phrase)

### Using textrank

In [ ]:
#conda install --upgrade scipy networkx


In [ ]:
def keyphrase_textrank(phrase):
    import spacy
    import pytextrank
    # load a french model
    nlp = spacy.load("en_core_web_sm")
    # add pytextrank to the pipe
    nlp.add_pipe("textrank")
    doc = nlp(phrase)
    textrank_keyphrases = doc._.phrases
    print(textrank_keyphrases[0].text)

In [ ]:
#keyphrase_textrank(phrase)

### Using YAKE

In [ ]:
#pip install git+https://github.com/LIAAD/yake

In [ ]:
from yake import KeywordExtractor as Yake
yake = Yake(lan="en")
yake_keyphrases = yake.extract_keywords(phrase)
print(yake_keyphrases)

In [ ]:
def keyphrase_yake(phrase,word_dict):
    from yake import KeywordExtractor as Yake
    yake = Yake(lan="en")
    yake_keyphrases = yake.extract_keywords(phrase)
    #print(yake_keyphrases)
    import yake
    
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(phrase)
    language = "en"
    max_ngram_size = 6
    deduplication_threshold = 0.8
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 1

    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(phrase)

    for kw in keywords:
        print(kw[0])
    keyphrase = kw[0]
    keyphr = keyphrase.lower()
    keyph = keyphr.split(' ')
    stime = word_dict[keyph[0]][0]
    etime = word_dict[keyph[-1]][1]
    key=keyphrase.upper()
    #print("")
    return key, stime, etime
    #print("=======================")
#     print(f"Keyphrase starting time: {stime} seconds")
#     print(f"Keyphrase ending time: {etime} seconds")

In [ ]:
#keyphrase_yake(phrase)[0]

In [ ]:
# keyphr = keyphrase.lower()
# keyph = keyphr.split(' ')
# stime = word_dict[keyph[0]][0]
# etime = word_dict[keyph[-1]][1]
# print("")
# print(keyphrase.upper())
# print("=======================")
# print(f"Keyphrase starting time: {stime} seconds")
# print(f"Keyphrase ending time: {etime} seconds")
    


### Using SUMMA

In [ ]:

#pip install summa keybert

In [ ]:
from summa import keywords
TR_keywords = keywords.keywords(phrase, scores=True)
print(TR_keywords[0:15])

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')

In [ ]:
#pickle.dump(kw_model, open('kw_model.pkl','wb'))

In [ ]:
def keyphrase_suma(phrase,word_dict):
    from summa import keywords
    TR_keywords = keywords.keywords(phrase, scores=True)
    print(TR_keywords[0:15])

    from keybert import KeyBERT
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    #kw_model = pickle.load(open('kw_model.pkl','rb'))
    keywords = kw_model.extract_keywords(phrase, 

                                         keyphrase_ngram_range=(1, 8), 

                                         stop_words='english', 

                                         highlight=False,

                                         top_n=1)

    keywords_list= list(dict(keywords).keys())

    keyphrase = keywords_list[0].lower()
    keyph = keyphrase.split(' ')
    stime = word_dict[keyph[0]][0]
    etime = word_dict[keyph[-1]][1]
    key=keyphrase.upper()
    #print("")
    return key, stime, etime
#     print(keyphrase.upper())
#     print("=======================")
#     print(f"Keyphrase starting time: {stime} seconds")
#     print(f"Keyphrase ending time: {etime} seconds")

In [ ]:
#keyphrase_suma(phrase)[0]

In [ ]:
# keyphrase = keywords_list[0].lower()
# keyph = keyphrase.split(' ')
# stime = word_dict[keyph[0]][0]
# etime = word_dict[keyph[-1]][1]
# print(keyphrase.upper())
# print("=======================")
# print(f"Keyphrase starting time: {stime} seconds")
# print(f"Keyphrase ending time: {etime} seconds")


### Using RAKE

In [ ]:
#!pip install rake-nltk

In [ ]:
def keyphrase_rake1(phrase,word_dict):
    from rake_nltk import Rake

    # Initialize the Rake object
    r = Rake()

    # Text to extract key phrases from
    text = phrase
    # Extract keywords
    r.extract_keywords_from_text(text)

    # Get the top 10 phrases
    results = r.get_ranked_phrases()[:5]

    # Print the phrases
    for keyphrase in results:
        try:

            keyphr = keyphrase.lower()
            keyph = keyphr.split(' ')
            stime = word_dict[keyph[0]][0]
            etime = word_dict[keyph[-1]][1]
            keyphrase=keyphrase.upper()
            #print("")
            return keyphrase, stime, etime
#             print("")
#             print(keyphrase.upper())
#             print("=======================")
#             print(f"Keyphrase starting time: {stime} seconds")
#             print(f"Keyphrase ending time: {etime} seconds")
        except:
            pass


In [ ]:
#keyphrase_rake1(phrase)

In [ ]:
# for keyphrase in results:
#     try:
        
#         keyphr = keyphrase.lower()
#         keyph = keyphr.split(' ')
#         stime = word_dict[keyph[0]][0]
#         etime = word_dict[keyph[-1]][1]
#         print("")
#         print(keyphrase.upper())
#         print("=======================")
#         print(f"Keyphrase starting time: {stime} seconds")
#         print(f"Keyphrase ending time: {etime} seconds")
#     except:
#         pass


In [ ]:
def keyphrase_rake2(phrase,word_dict):
    from rake_nltk import Rake
    import nltk
    nltk.download('stopwords')

    text = phrase
    # Create a Rake object and extract keywords
    rake_object = Rake(nltk.corpus.stopwords.words('english'))
    keyphrases = rake_object.extract_keywords_from_text(text)
    keyphrases = rake_object.get_ranked_phrases_with_scores()

    # Extract the top 5 keyphrases from the text
    result = [keyphrase[1] for keyphrase in keyphrases[:5]]

    # Print the extracted keyphrases
    print(result)
    keyphrases = result[2]

    for keyphrase in result:
        try:

            keyphr = keyphrase.lower()
            keyph = keyphr.split(' ')
            stime = word_dict[keyph[0]][0]
            etime = word_dict[keyph[-1]][1]
            keyphrase=keyphrase.upper()
            #print("")
            return keyphrase, stime, etime
#             print("")
#             print(keyphrase.upper())
#             print("=======================")
#             print(f"Keyphrase starting time: {stime} seconds")
#             print(f"Keyphrase ending time: {etime} seconds")
        except:
            pass


In [ ]:
#keyphrase_rake2(phrase)

In [ ]:
 
# for keyphrase in result:
#     try:
        
#         keyphr = keyphrase.lower()
#         keyph = keyphr.split(' ')
#         stime = word_dict[keyph[0]][0]
#         etime = word_dict[keyph[-1]][1]
#         print("")
#         print(keyphrase.upper())
#         print("=======================")
#         print(f"Keyphrase starting time: {stime} seconds")
#         print(f"Keyphrase ending time: {etime} seconds")
#     except:
#         pass
       


### Using SPAcy

In [ ]:
#!pip install spacy



In [ ]:
#pip install -U pip setuptools wheel

In [ ]:
#pip install -U spacy

In [ ]:
#!python -m spacy download en_core_web_sm

In [ ]:
!python -m spacy download en_core_web_sm


In [ ]:
#!conda install -c conda-forge g2p-en

In [ ]:
def keyphrase_spacy(phrase,word_dict):
    import spacy

    # Load the English language model
    nlp = spacy.load('en_core_web_sm')

    # Text to extract keywords from
    text = phrase
    # Process the text with spaCy
    doc = nlp(text)

    # Extract keywords
#     keywords = []
#     for token in doc:
#         if not token.is_stop and token.pos_ in ['NOUN', 'PROPN','ADJECTIVE']:
#             keywords.append(token.text)
    keyphrases = set()
    for chunk in doc.noun_chunks:
        if len(chunk) > 1 and chunk.root.pos_ != 'PRON':
            keyphrases.add(chunk.text)
    keyphrases = list(keyphrases)
    keyphrase = keyphrases[0]

    keyphr = keyphrase.lower()
    keyph = keyphr.split(' ')
    stime = word_dict[keyph[0]][0]
    etime = word_dict[keyph[-1]][1]
    keyphrase=keyphrase.upper()
    #print("")
    return keyphrase, stime, etime
#     return keyphrases[0]



In [ ]:
#keyphrase_spacy(phrase)

### Using TF IDF Vectorizer

In [ ]:
def keyphrase_tfidf(phrase):
    from sklearn.feature_extraction.text import TfidfVectorizer

    # Create a TfidfVectorizer object and fit it to the text
    vectorizer = TfidfVectorizer(ngram_range=(1,5), max_df=0.1, stop_words='english')
    phra = list(phrase.split(' '))
    vectorizer.fit(phra)

    # Extract the top 5 keyphrases from the text
    features = vectorizer.transform(phra)
    scores = zip(vectorizer.get_feature_names_out(), features.sum(axis=0).tolist()[0])
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    result = [item[0] for item in sorted_scores[:10]]

    # Print the extracted keyphrases
    print(result)



In [ ]:
keyphrase_tfidf(phrase)

## Sentiment Analysis using FLAIR

In [ ]:
#pip install flair

In [ ]:
#!pip install --user flair

In [ ]:
#conda install pandas

In [ ]:
import pandas as pd
text = phrase.split('.')
df = pd.DataFrame([phrase])
df.rename(columns = {0:'text'},inplace = True)

In [ ]:
df.head()

In [ ]:
def detect_sentiment(phrase):
    import pandas as pd
    text = phrase.split('.')
    df = pd.DataFrame([phrase])
    df.rename(columns = {0:'text'},inplace = True)
    from flair.models import TextClassifier
    from flair.data import Sentence
    sia = TextClassifier.load('en-sentiment')
    
def flair_prediction(x):
    from flair.models import TextClassifier
    from flair.data import Sentence
    sia = TextClassifier.load('en-sentiment')
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    if "POSITIVE" in str(score):
        return "Positive Sentiment"
    elif "NEGATIVE" in str(score):
        return "Negative Sentiment"
    else:
        return "Neutral Sentiment"
    df["sentiment"] = df['text'].apply(flair_prediction)
    sentiment = df["sentiment"][0]
    return sentiment


In [ ]:
# from flair.models import TextClassifier
# from flair.data import Sentence
# classifier = TextClassifier.load('en-sentiment')
# sentence = Sentence('I love Flair!')
# classifier.predict(sentence)
# label = sentence.sentiment.label
# score = sentence.sentiment.score
# from flair.models import TextClassifier
# from flair.data import Sentence

# # load the sentiment classifier
# classifier = TextClassifier.load('en-sentiment')

# # create a sentence object
# sentence = Sentence('I love Flair!')

# # predict the sentiment
# classifier.predict(sentence)

# # get the sentiment label and score
# label = sentence.sentiment.label
# score = sentence.sentiment.score

# print(label)
# print(score)


In [ ]:
#pip install vaderSentiment


### Sentiment analysis using Vader

In [1]:
%%writefile vader_sentiment.py
def vader_sentiment(phrase):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()
    text = phrase
    scores = analyzer.polarity_scores(text)
    return scores



Writing vader_sentiment.py


In [ ]:
#flair_prediction(phrase)

In [ ]:
#sentiment = df["sentiment"][0]

In [ ]:
#print(sentiment)

#### Lets check the accuracy of this Flair model using a Kaggle dataset of movie reviews

In [ ]:
test_df = pd.read_csv("Test.csv")

In [ ]:
test_df.head()

In [ ]:
#test_df["sentiment"] = test_df['text'].apply(flair_prediction)

In [ ]:
test_df['sentiment'].unique()

In [ ]:
new_df = test_df.copy()
new_df['sentiment']=new_df['sentiment'].map({'Positive Sentiment':1,'Negative Sentiment':0})
new_df.head()

In [ ]:
y_test = new_df['label']
y_predicted = new_df['sentiment']

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, average_precision_score,auc,ConfusionMatrixDisplay

In [ ]:
print(f"Prediction: Accuracy of FLAIR model:{accuracy_score(y_test,y_predicted):.3f}")
print(f"Prediction: Recall score of FLAIR model:{recall_score(y_test,y_predicted):.3f}")
print(f"Prediction: Precision score of FLAIR model:{precision_score(y_test,y_predicted):.3f}")
print(f"Prediction: F1 score of FLAIR model:{f1_score(y_test,y_predicted):.3f}")

In [ ]:
from matplotlib.ticker import StrMethodFormatter
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})
test_cm = confusion_matrix(y_test, y_predicted)
fig, (ax) = plt.subplots(1,figsize=(8,6))
ConfusionMatrixDisplay(test_cm).plot(ax=ax)
ax.set_title("Confusion Matrix for FLAIR model")
ax.grid(False)
plt.show()

In [ ]:
def create_textoutput_keyphrase_sentiment(video):
    original_stdout = sys.stdout # Save a reference to the original standard output
    for name in vid_list:
        if name in video:
            f = str(name)
    filename = "C:/Users/manas_nmr2rze/Desktop/Capstone/Output/keyphrase/"+f+".txt" 
#     video_link = video
    phrase, word_dict = assemblyai_text_corpus(video)

    with open(filename, 'w') as f:
        sys.stdout = f # Change the standard output to the file we created.
        print(f"keyphrase and sentiment analysis of video: {f}")
#         words, word_dict,audio_text=local_google_transcribe(video)

        print("----------Sentiment of video----------------")
        print(vader_sentiment(phrase))
        print("----------KeyPhrase of video----------------")
        print("Keyphrase using OpenAI:")
        print(keyphrase_openai(phrase))
        print("Keyphrase using YAKE:")
        print(keyphrase_yake(phrase,word_dict))
        print("")
        print("Keyphrase using SUMA:")
        print(keyphrase_suma(phrase,word_dict))
        print("")
        print("Keyphrase using RAKE:")
        print(keyphrase_rake1(phrase,word_dict))
        print("")
        print("Keyphrase using SpaCy:")
        print(keyphrase_spacy(phrase,word_dict))
              
        
        sys.stdout = original_stdout 
    

In [ ]:
vid_link = (str(link_list[10]))
create_textoutput_keyphrase_sentiment(vid_link)

In [ ]:
for i in link_list[:1]:
    create_textoutput_keyphrase_sentiment(i)    

In [ ]:
video1 = (link_list[1])

words, word_dict,aud_text=local_google_transcribe(video1)
print(aud_text)
ph = key_phrase(video1)
print(ph)
phras = aud_text+ph
print("----------Sentiment of video----------------")
print(vader_sentiment(phras))
print("----------KeyPhrase of video----------------")
print("Keyphrase using OpenAI:")
print(keyphrase_openai(phras))
print("Keyphrase using YAKE:")
print(keyphrase_yake(phras,word_dict))
print("Keyphrase using SUMA:")
print(keyphrase_suma(phras,word_dict))
print("Keyphrase using RAKE:")
print(keyphrase_rake1(phras,word_dict))
print("Keyphrase using SpaCy:")
print(keyphrase_spacy(phras,word_dict))

In [ ]:
conda deactivate

# Sound Analysis

In [ ]:
#!pip install python-utils
#!pip install utils
#!pip install librosa soundfile
#!pip install pyaudio wave

In [ ]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result


def load_data(test_size=0.2):
    X, y = [], []
    for file in glob.glob("data/Actor_*/*.wav"):
        # get the base name of the audio file
        basename = os.path.basename(file)
        # get the emotion label
        emotion = int2emotion[basename.split("-")[2]]
        # we allow only AVAILABLE_EMOTIONS we set
        if emotion not in AVAILABLE_EMOTIONS:
            continue
        # extract speech features
        features = extract_feature(file, mfcc=True, chroma=True, mel=True)
        # add to data
        X.append(features)
        y.append(emotion)
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [ ]:
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
#from utils import load_data

import os
import pickle

# load RAVDESS dataset
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("[+] Number of features:", X_train.shape[1])
# best model, determined by a grid search
model_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
model = MLPClassifier(**model_params)

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

# now we save the model
# make result directory if doesn't exist yet
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))

## detect call to actions

In [ ]:
%writefile call_to_action
def call_to_action(video_uri):
    import re

    # Define a list of call to action phrases
    call_to_action_phrases = [
        'buy now', 'order now', 'subscribe today', 'register now',
        'download your free guide', 'limited time offer', 'sign up now',
        'shop now', 'get started', 'join now', 'claim your free trial',
        'book your appointment', 'apply now', 'enroll today', 'try it now'
    ]

    # Define a regular expression pattern that matches any of the call to action phrases
    pattern = r'\b(?:%s)\b' % '|'.join(call_to_action_phrases)

    # Define a sample text
    text = "Don't miss out on our limited time offer! Buy now and get your free guide."

    # Find all instances of call to action phrases in the text
    matches = re.findall(pattern, text, flags=re.IGNORECASE)

    # Print the matches
    print(matches)


### Creating JSON

In [2]:
import json

aDict = {"a":54, "b":87}
jsonString = json.dumps(aDict)
jsonFile = open("data.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

#### importing and reading json file

In [3]:
import json

with open('data.json', 'r') as json_file:
    json_object = json.load(json_file)

print(json_object)

print(json.dumps(json_object))

print(json.dumps(json_object, indent=1))

{'a': 54, 'b': 87}
{"a": 54, "b": 87}
{
 "a": 54,
 "b": 87
}


In [10]:
import json

def my_function(name, age, city):
    my_dict = {"name": name, "age": age, "city": city}
    my_json = json.dumps(my_dict)
    return my_json


In [12]:
my_function('name', 50, 'city')

'{"name": "name", "age": 50, "city": "city"}'